<div style="text-align: center;">
    <h1 style="color:black;" > ANEMIA EN NIÑOS ENTRE 6 Y 36 MESES EN EL PERÚ <a id="indice"></a> </h1>
</div>

<h1 style="color:black;" > Importación de librerías <a id="1"></a> </h1>

Se importan las librerías necesarias:

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os

!pip install missingno
import missingno
!pip install upsetplot pyjanitor==0.23.1
import upsetplot
import janitor
import scipy.stats

from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, MinMaxScaler, RobustScaler
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.decomposition import PCA

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

!pip install tensorflow
!pip install visualkeras
import visualkeras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import confusion_matrix , classification_report
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

<div style="text-align: left; margin-top: 2rem;">
  <button style="padding: 10px 20px; font-size: 12px; cursor: pointer; background-color: #f0f0f0; color: white; border: none;">
    <a href="#indice" style="color: black; text-decoration: none;">Regresar Tabla de contenido</a><br>
  </button>
</div>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<h1 style="color:black;" > Extracción de datos <a id="3"></a> </h1>

Accedemos y leemos el archivo que está almacenado en nuestro entorno de trabajo, el cual contiene el conjunto de datos:

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Coding Dojo/Proyecto/datos_limpios.csv')

In [ ]:
df.head()

,n_personas_hogar,n_niños_menores_cinco,área_residencia,lugar_residencia,fuente_agua_para_beber,piso_vivienda,índice_riqueza,puntuación_índice_riqueza,seguro_salud_essalud/ipss,seguro_salud_militar,seguro_salud_integral,seguro_salud_compañía_seguros,seguro_salud_seguro_privado,seguro_salud_no_tiene,n_meses_lactancia,lugar_parto,tamaño_niño_al_nacer,n_veces_comida_sólida_semisólida_día_anterior,niño_afiliado_sis,durante_embarazo_diagnóstico_anemia,diarrea_últimas_dos_semanas,fiebre_últimas_dos_semanas,tos_últimas_dos_semanas,suplemento_vitamina_a_reciente,antiparasitarios_últimos_seis_meses,peso_kg,talla_cm,nivel_hemoglobina_ajustada_g/dl,nivel_anemia,hierro_jarabe_última_semana,hierro_polvo_micronutrientes_última_semana,hierro_gotas_última_semana,hierro_otro_última_semana,edad_meses,sexo_niños,nivel_educativo_madre,n_orden_nacimiento,b_té_café,b_leche_materna_refrigerada_conservada,b_leche_polvo_nam_s26_similac,b_otra_leche_fresca_evaporada_polvo,f_naranjas_mandarina_lima_maracuyá_toronja,f_secas,harina_pan_fideos_galletas_otra_comida_con_cereales,papa_yuca_olluco_oca_mashua,huevos,res_pollo_pescado_mariscos_otrascarnes_hígado_mondongo_otrasvísceras,v_camote_zanahorias_zapallo,v_hoja_verdeoscuro_espinaca_acelga_otros,f_mango_papaya_aguaje_otrasfrutas_vitamina_a,f_plátano_isla_seda_maduro_manzana_palta_otras,habas_frijol_lenteja_soya_pallares_garbanzo_arvejas,queso_yogurt_otros_lácteos,comida_hecha_aceite_grasas_mantequilla,nivel_educativo_esposo_compañero_merge5,actualmente_trabajando,nivel_educativo_esposo_compañero_2,violencia_emocional,violencia_física_menos_severa,violencia_física_severa,violencia_sexual,edad_madre,imc_madre,actualmente_embarazada,nivel_hemoglobina_ajustada_g/dl_madre,nivel_anemia_madre,región_natural,n_veces_lavado_manos,lavado_manos_después_baño,lavado_manos_después_pañales,lavado_manos_antes_preparar_comida,lavado_manos_antes_alimentar_niño,persona_alimenta_niño,nivel_educativo_esposo_compañero_REC91_22,preparación_alimentos_minsa_último_año,n_veces_preparación_alimentos_minsa_último_año,visita_minsa_alimentación_último_año,n_veces_visita_minsa_alimentación_último_año,seguro_salud_público,seguro_salud_privado,amamantamiento,amamantamiento2,pobreza,fuente_agua,material_piso,tamaño_hogar,menores_cinco_años,edad_niño,edad_niño2,orden_nacimiento,orden_nacimiento2,edad_madre2,edad_madre3,parto_institución_salud,consumo_leche,productos_lácteos,frutas,diversidad_dietética,diversidad_dietética2,hierro_última_semana,anemia
0,3.0,1.0,1.0,1.0,11,33,5,1.469055,0.0,1.0,0.0,0.0,0.0,0.0,12.0,31.0,2.0,4,0.0,2,0,0,2,0,1,15.8,94.1,11.6,4,2,2,2,2,31.0,1.0,3.0,1.0,1,0,1,0,1,0,1,1,1,1,0,0,1,1,1,0,1,3,0.0,5,0,0,0,0,22.0,2636.0,0.0,12.3,4.0,3.0,4,1,0,1,1,1,3,2,0.0,2,0.0,1.0,0.0,1,2,0,1,1,0,0,2,4,1,0,0,0,1,1,1,1,6,1,0,0
1,4.0,1.0,1.0,1.0,11,34,3,0.484814,0.0,0.0,1.0,0.0,0.0,0.0,22.0,21.0,2.0,4,1.0,2,0,0,0,1,0,12.2,86.0,12.4,4,2,2,2,2,28.0,1.0,2.0,3.0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,3,1.0,5,1,0,0,0,34.0,3065.0,0.0,15.0,4.0,3.0,4,1,0,0,0,1,3,2,0.0,2,0.0,1.0,0.0,1,2,0,1,1,0,0,2,3,3,1,2,1,1,1,1,1,4,1,0,0
2,3.0,1.0,1.0,1.0,11,34,4,0.947885,1.0,0.0,0.0,0.0,0.0,0.0,16.0,22.0,3.0,4,0.0,2,0,1,2,0,1,12.6,94.0,11.7,4,2,2,2,2,33.0,2.0,3.0,1.0,0,0,0,1,1,0,0,1,1,1,1,1,0,0,0,0,1,3,1.0,5,0,0,0,0,32.0,2239.0,0.0,12.6,4.0,3.0,4,1,0,1,0,1,4,2,0.0,2,0.0,1.0,0.0,1,2,0,1,1,0,0,2,4,1,0,2,1,1,1,1,1,6,1,0,0
3,5.0,1.0,1.0,1.0,11,34,2,0.214832,0.0,0.0,1.0,0.0,0.0,0.0,6.0,21.0,3.0,3,1.0,2,0,0,0,0,0,8.3,65.6,10.4,3,2,2,1,2,6.0,1.0,2.0,1.0,0,0,0,0,0,0,1,1,0,1,1,1,0,1,0,0,0,2,1.0,4,0,0,0,0,31.0,2476.0,0.0,14.5,4.0,3.0,4,1,1,1,0,1,2,2,0.0,2,0.0,1.0,0.0,2,3,1,1,1,1,0,0,0,1,0,2,1,1,0,0,1,6,1,1,1
4,4.0,1.0,1.0,1.0,11,34,4,0.956133,1.0,0.0,0.0,0.0,0.0,0.0,14.0,22.0,3.0,5,0.0,2,0,0,0,0,0,12.3,81.3,11.4,4,2,2,2,2,18.0,1.0,3.0,2.0,1,0,0,1,0,0,1,1,0,1,0,1,0,1,1,0,1,2,1.0,4,0,1,0,0,33.0,2810.0,0.0,13.9,4.0,3.0,4,1,0,1,0,1,2,2,0.0,2,0.0,1.0,0.0,1,2,0,1,1,0,0,1,2,2,1,2,1,1,1,1,1,7,1,0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9205 entries, 0 to 9204
Columns: 101 entries, n_personas_hogar to anemia
dtypes: float64(33), int64(68)
memory usage: 7.1 MB


<h1 style="color:black;" > 6. Elección de características más relacionadas <a id="7"></a> </h1>

<h2 style="color:black;" > 6.1. Variable objetivo categórica multiclase <a id="7"></a> </h2>

In [ ]:
matriz_correlación_multiclase1 = df.corr()

filtro_variables_multiclase1 = {
    variable: matriz_correlación_multiclase1['nivel_anemia'][variable]
    for variable in matriz_correlación_multiclase1.index
    if abs(matriz_correlación_multiclase1['nivel_anemia'][variable]) >= 0.00
}

filtro_ordenado_multiclase1 = dict(sorted(filtro_variables_multiclase1.items(), key=lambda item: abs(item[1]), reverse=True))

print(f"Número de variables: {len(filtro_ordenado_multiclase1)}")
print("Variables con correlación mayor e igual al 0% con 'nivel_anemia':")
for variable, correlación in filtro_ordenado_multiclase1.items():
    print(f"{variable}: {correlación:.3f}")

Número de variables: 101
Variables con correlación mayor e igual al 0% con 'nivel_anemia':
nivel_anemia: 1.000
anemia: -0.899
nivel_hemoglobina_ajustada_g/dl: 0.866
talla_cm: 0.337
peso_kg: 0.313
edad_niño2: 0.278
edad_meses: 0.271
edad_niño: 0.267
puntuación_índice_riqueza: 0.231
índice_riqueza: 0.227
pobreza: -0.210
amamantamiento: -0.203
amamantamiento2: -0.200
nivel_hemoglobina_ajustada_g/dl_madre: 0.193
región_natural: -0.173
piso_vivienda: 0.167
nivel_anemia_madre: 0.163
n_meses_lactancia: 0.151
material_piso: 0.148
productos_lácteos: 0.144
consumo_leche: 0.134
área_residencia: -0.129
n_veces_comida_sólida_semisólida_día_anterior: 0.127
lugar_residencia: -0.125
antiparasitarios_últimos_seis_meses: 0.120
n_niños_menores_cinco: -0.109
diversidad_dietética2: 0.108
hierro_gotas_última_semana: 0.102
diversidad_dietética: 0.102
nivel_educativo_esposo_compañero_REC91_22: 0.096
edad_madre: 0.094
nivel_educativo_esposo_compañero_merge5: 0.094
durante_embarazo_diagnóstico_anemia: 0.093
ord

In [ ]:
filtro_nombres_multiclase1 = list(filtro_ordenado_multiclase1.keys())
filtro_nombres_multiclase1

['nivel_anemia',
 'anemia',
 'nivel_hemoglobina_ajustada_g/dl',
 'talla_cm',
 'peso_kg',
 'edad_niño2',
 'edad_meses',
 'edad_niño',
 'puntuación_índice_riqueza',
 'índice_riqueza',
 'pobreza',
 'amamantamiento',
 'amamantamiento2',
 'nivel_hemoglobina_ajustada_g/dl_madre',
 'región_natural',
 'piso_vivienda',
 'nivel_anemia_madre',
 'n_meses_lactancia',
 'material_piso',
 'productos_lácteos',
 'consumo_leche',
 'área_residencia',
 'n_veces_comida_sólida_semisólida_día_anterior',
 'lugar_residencia',
 'antiparasitarios_últimos_seis_meses',
 'n_niños_menores_cinco',
 'diversidad_dietética2',
 'hierro_gotas_última_semana',
 'diversidad_dietética',
 'nivel_educativo_esposo_compañero_REC91_22',
 'edad_madre',
 'nivel_educativo_esposo_compañero_merge5',
 'durante_embarazo_diagnóstico_anemia',
 'orden_nacimiento',
 'hierro_última_semana',
 'seguro_salud_essalud/ipss',
 'niño_afiliado_sis',
 'menores_cinco_años',
 'persona_alimenta_niño',
 'nivel_educativo_esposo_compañero_2',
 'n_veces_lavad

In [ ]:
df_multiclase_filtro1 = df[filtro_nombres_multiclase1]
df_multiclase_filtro1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9205 entries, 0 to 9204
Columns: 101 entries, nivel_anemia to hierro_otro_última_semana
dtypes: float64(33), int64(68)
memory usage: 7.1 MB


In [ ]:
df_multiclase_filtro1 = df_multiclase_filtro1.drop(['nivel_hemoglobina_ajustada_g/dl',
                                                'anemia',
                                                'edad_niño',
                                                'edad_meses',
                                                'índice_riqueza',
                                                'pobreza',
                                                'amamantamiento2',
                                                'nivel_anemia_madre',
                                                'material_piso',
                                                'n_meses_lactancia',
                                                'consumo_leche',
                                                'lugar_residencia',
                                                'diversidad_dietética',
                                                'nivel_educativo_esposo_compañero_merge5',
                                                'hierro_última_semana',
                                                'niño_afiliado_sis',
                                                'menores_cinco_años',
                                                'nivel_educativo_esposo_compañero_2',
                                                'seguro_salud_integral',
                                                'b_otra_leche_fresca_evaporada_polvo',
                                                'n_orden_nacimiento',
                                                'b_leche_polvo_nam_s26_similac',
                                                'orden_nacimiento2',
                                                'queso_yogurt_otros_lácteos',
                                                'edad_madre3',
                                                'edad_madre2',
                                                'lavado_manos_antes_alimentar_niño',
                                                'parto_institución_salud',
                                                'seguro_salud_privado',
                                                'tamaño_hogar',
                                                'lavado_manos_después_baño',
                                                'seguro_salud_compañía_seguros',
                                                'hierro_jarabe_última_semana',
                                                'violencia_física_menos_severa',
                                                'lavado_manos_antes_preparar_comida',
                                                'seguro_salud_militar',
                                                'seguro_salud_público',
                                                'seguro_salud_seguro_privado',
                                                'fuente_agua_para_beber',
                                                'seguro_salud_no_tiene',
                                                'b_leche_materna_refrigerada_conservada',
                                                'hierro_polvo_micronutrientes_última_semana',
                                                'lavado_manos_después_pañales',
                                                'violencia_emocional',
                                                'violencia_sexual',
                                                'hierro_otro_última_semana'], axis = 1)
df_multiclase_filtro1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9205 entries, 0 to 9204
Data columns (total 55 columns):
 #   Column                                                                Non-Null Count  Dtype  
---  ------                                                                --------------  -----  
 0   nivel_anemia                                                          9205 non-null   int64  
 1   talla_cm                                                              9205 non-null   float64
 2   peso_kg                                                               9205 non-null   float64
 3   edad_niño2                                                            9205 non-null   int64  
 4   puntuación_índice_riqueza                                             9205 non-null   float64
 5   amamantamiento                                                        9205 non-null   int64  
 6   nivel_hemoglobina_ajustada_g/dl_madre                                 9205 non-null   float64
 7

In [ ]:
y2 = df_multiclase_filtro1['nivel_anemia']
y2.shape

(9205,)

In [ ]:
X2 = df_multiclase_filtro1.drop('nivel_anemia', axis = 1)
X2.shape

(9205, 54)

In [ ]:
matriz_correlación_multiclase2 = df_multiclase_filtro1.corr()

filtro_variables_multiclase2 = {
    variable: matriz_correlación_multiclase2['nivel_anemia'][variable]
    for variable in matriz_correlación_multiclase2.index
    if abs(matriz_correlación_multiclase2['nivel_anemia'][variable]) >= 0.07
}

filtro_ordenado_multiclase2 = dict(sorted(filtro_variables_multiclase2.items(), key=lambda item: abs(item[1]), reverse=True))

print(f"Número de variables: {len(filtro_ordenado_multiclase2)}")
print("Variables con correlación mayor e igual al 3% con 'nivel_anemia':")
for variable, correlación in filtro_ordenado_multiclase2.items():
    print(f"{variable}: {correlación:.3f}")

Número de variables: 27
Variables con correlación mayor e igual al 3% con 'nivel_anemia':
nivel_anemia: 1.000
talla_cm: 0.337
peso_kg: 0.313
edad_niño2: 0.278
puntuación_índice_riqueza: 0.231
amamantamiento: -0.203
nivel_hemoglobina_ajustada_g/dl_madre: 0.193
región_natural: -0.173
piso_vivienda: 0.167
productos_lácteos: 0.144
área_residencia: -0.129
n_veces_comida_sólida_semisólida_día_anterior: 0.127
antiparasitarios_últimos_seis_meses: 0.120
n_niños_menores_cinco: -0.109
diversidad_dietética2: 0.108
hierro_gotas_última_semana: 0.102
nivel_educativo_esposo_compañero_REC91_22: 0.096
edad_madre: 0.094
durante_embarazo_diagnóstico_anemia: 0.093
orden_nacimiento: -0.092
seguro_salud_essalud/ipss: 0.091
persona_alimenta_niño: 0.090
n_veces_lavado_manos: 0.089
nivel_educativo_madre: 0.087
fiebre_últimas_dos_semanas: -0.087
lugar_parto: 0.086
n_personas_hogar: -0.076


In [ ]:
filtro_nombres_multiclase2 = list(filtro_ordenado_multiclase2.keys())
filtro_nombres_multiclase2

['nivel_anemia',
 'talla_cm',
 'peso_kg',
 'edad_niño2',
 'puntuación_índice_riqueza',
 'amamantamiento',
 'nivel_hemoglobina_ajustada_g/dl_madre',
 'región_natural',
 'piso_vivienda',
 'productos_lácteos',
 'área_residencia',
 'n_veces_comida_sólida_semisólida_día_anterior',
 'antiparasitarios_últimos_seis_meses',
 'n_niños_menores_cinco',
 'diversidad_dietética2',
 'hierro_gotas_última_semana',
 'nivel_educativo_esposo_compañero_REC91_22',
 'edad_madre',
 'durante_embarazo_diagnóstico_anemia',
 'orden_nacimiento',
 'seguro_salud_essalud/ipss',
 'persona_alimenta_niño',
 'n_veces_lavado_manos',
 'nivel_educativo_madre',
 'fiebre_últimas_dos_semanas',
 'lugar_parto',
 'n_personas_hogar']

In [ ]:
df_multiclase_filtro2 = df[filtro_nombres_multiclase2]
df_multiclase_filtro2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9205 entries, 0 to 9204
Data columns (total 27 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   nivel_anemia                                   9205 non-null   int64  
 1   talla_cm                                       9205 non-null   float64
 2   peso_kg                                        9205 non-null   float64
 3   edad_niño2                                     9205 non-null   int64  
 4   puntuación_índice_riqueza                      9205 non-null   float64
 5   amamantamiento                                 9205 non-null   int64  
 6   nivel_hemoglobina_ajustada_g/dl_madre          9205 non-null   float64
 7   región_natural                                 9205 non-null   float64
 8   piso_vivienda                                  9205 non-null   int64  
 9   productos_lácteos                              9205 

In [ ]:
y2_filtro = df_multiclase_filtro2['nivel_anemia']
y2_filtro.shape

(9205,)

In [ ]:
X2_filtro = df_multiclase_filtro2.drop('nivel_anemia', axis = 1)
X2_filtro.shape

(9205, 26)

<h2 style="color:black;" > 6.2. Variable objetivo categórica binaria <a id="7"></a> </h2>

In [ ]:
matriz_correlación_binaria1 = df.corr()

filtro_variables_binaria1 = {
    variable: matriz_correlación_binaria1['anemia'][variable]
    for variable in matriz_correlación_binaria1.index
    if abs(matriz_correlación_binaria1['anemia'][variable]) >= 0.00
}

filtro_ordenado_binaria1 = dict(sorted(filtro_variables_binaria1.items(), key=lambda item: abs(item[1]), reverse=True))

print(f"Número de variables: {len(filtro_ordenado_binaria1)}")
print("Variables con correlación mayor e igual al 0% con 'anemia':")
for variable, correlación in filtro_ordenado_binaria1.items():
    print(f"{variable}: {correlación:.3f}")

Número de variables: 101
Variables con correlación mayor e igual al 0% con 'anemia':
anemia: 1.000
nivel_anemia: -0.899
nivel_hemoglobina_ajustada_g/dl: -0.775
talla_cm: -0.304
peso_kg: -0.281
edad_niño2: -0.255
edad_meses: -0.249
edad_niño: -0.240
puntuación_índice_riqueza: -0.204
índice_riqueza: -0.201
pobreza: 0.191
nivel_hemoglobina_ajustada_g/dl_madre: -0.182
amamantamiento: 0.177
amamantamiento2: 0.176
región_natural: 0.161
n_meses_lactancia: -0.143
piso_vivienda: -0.139
nivel_anemia_madre: -0.139
productos_lácteos: -0.120
material_piso: -0.117
consumo_leche: -0.112
n_veces_comida_sólida_semisólida_día_anterior: -0.111
antiparasitarios_últimos_seis_meses: -0.109
área_residencia: 0.106
lugar_residencia: 0.103
n_niños_menores_cinco: 0.096
nivel_educativo_esposo_compañero_REC91_22: -0.095
edad_madre: -0.092
lugar_parto: -0.090
menores_cinco_años: 0.088
diversidad_dietética2: -0.087
hierro_gotas_última_semana: -0.087
nivel_educativo_esposo_compañero_merge5: -0.086
nivel_educativo_esp

In [ ]:
filtro_nombres_binaria1 = list(filtro_ordenado_binaria1.keys())
filtro_nombres_binaria1

['anemia',
 'nivel_anemia',
 'nivel_hemoglobina_ajustada_g/dl',
 'talla_cm',
 'peso_kg',
 'edad_niño2',
 'edad_meses',
 'edad_niño',
 'puntuación_índice_riqueza',
 'índice_riqueza',
 'pobreza',
 'nivel_hemoglobina_ajustada_g/dl_madre',
 'amamantamiento',
 'amamantamiento2',
 'región_natural',
 'n_meses_lactancia',
 'piso_vivienda',
 'nivel_anemia_madre',
 'productos_lácteos',
 'material_piso',
 'consumo_leche',
 'n_veces_comida_sólida_semisólida_día_anterior',
 'antiparasitarios_últimos_seis_meses',
 'área_residencia',
 'lugar_residencia',
 'n_niños_menores_cinco',
 'nivel_educativo_esposo_compañero_REC91_22',
 'edad_madre',
 'lugar_parto',
 'menores_cinco_años',
 'diversidad_dietética2',
 'hierro_gotas_última_semana',
 'nivel_educativo_esposo_compañero_merge5',
 'nivel_educativo_esposo_compañero_2',
 'niño_afiliado_sis',
 'hierro_última_semana',
 'edad_madre2',
 'persona_alimenta_niño',
 'seguro_salud_essalud/ipss',
 'seguro_salud_integral',
 'orden_nacimiento',
 'queso_yogurt_otros_l

In [ ]:
df_binaria_filtro1 = df[filtro_nombres_binaria1]
df_binaria_filtro1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9205 entries, 0 to 9204
Columns: 101 entries, anemia to b_té_café
dtypes: float64(33), int64(68)
memory usage: 7.1 MB


In [ ]:
df_binaria_filtro1 = df_binaria_filtro1.drop(['nivel_hemoglobina_ajustada_g/dl',
                                                'nivel_anemia',
                                                'edad_niño',
                                                'edad_meses',
                                                'índice_riqueza',
                                                'pobreza',
                                                'amamantamiento2',
                                                'nivel_anemia_madre',
                                                'material_piso',
                                                'n_meses_lactancia',
                                                'consumo_leche',
                                                'lugar_residencia',
                                                'diversidad_dietética',
                                                'nivel_educativo_esposo_compañero_merge5',
                                                'hierro_última_semana',
                                                'seguro_salud_essalud/ipss',
                                                'menores_cinco_años',
                                                'nivel_educativo_esposo_compañero_2',
                                                'seguro_salud_integral',
                                                'b_otra_leche_fresca_evaporada_polvo',
                                                'n_orden_nacimiento',
                                                'b_leche_polvo_nam_s26_similac',
                                                'orden_nacimiento2',
                                                'queso_yogurt_otros_lácteos',
                                                'edad_madre3',
                                                'edad_madre2',
                                                'lavado_manos_antes_alimentar_niño',
                                                'parto_institución_salud',
                                                'seguro_salud_privado',
                                                'tamaño_hogar',
                                                'lavado_manos_después_baño',
                                                'seguro_salud_compañía_seguros',
                                                'hierro_jarabe_última_semana',
                                                'violencia_física_menos_severa',
                                                'lavado_manos_antes_preparar_comida',
                                                'seguro_salud_militar',
                                                'seguro_salud_público',
                                                'seguro_salud_seguro_privado',
                                                'fuente_agua_para_beber',
                                                'seguro_salud_no_tiene',
                                                'b_leche_materna_refrigerada_conservada',
                                                'hierro_polvo_micronutrientes_última_semana',
                                                'lavado_manos_después_pañales',
                                                'violencia_emocional',
                                                'violencia_sexual',
                                                'hierro_otro_última_semana'], axis = 1)
df_binaria_filtro1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9205 entries, 0 to 9204
Data columns (total 55 columns):
 #   Column                                                                Non-Null Count  Dtype  
---  ------                                                                --------------  -----  
 0   anemia                                                                9205 non-null   int64  
 1   talla_cm                                                              9205 non-null   float64
 2   peso_kg                                                               9205 non-null   float64
 3   edad_niño2                                                            9205 non-null   int64  
 4   puntuación_índice_riqueza                                             9205 non-null   float64
 5   nivel_hemoglobina_ajustada_g/dl_madre                                 9205 non-null   float64
 6   amamantamiento                                                        9205 non-null   int64  
 7

In [ ]:
y3 = df_binaria_filtro1['anemia']
y3.shape

(9205,)

In [ ]:
X3 = df_binaria_filtro1.drop('anemia', axis = 1)
X3.shape

(9205, 54)

In [ ]:
matriz_correlación_binaria2 = df_binaria_filtro1.corr()

filtro_variables_binaria2 = {
    variable: matriz_correlación_binaria2['anemia'][variable]
    for variable in matriz_correlación_binaria2.index
    if abs(matriz_correlación_binaria2['anemia'][variable]) >= 0.07
}

filtro_ordenado_binaria2 = dict(sorted(filtro_variables_binaria2.items(), key=lambda item: abs(item[1]), reverse=True))

print(f"Número de variables: {len(filtro_ordenado_binaria2)}")
print("Variables con correlación mayor e igual al 7% con 'anemia':")
for variable, correlación in filtro_ordenado_binaria2.items():
    print(f"{variable}: {correlación:.3f}")

Número de variables: 25
Variables con correlación mayor e igual al 7% con 'anemia':
anemia: 1.000
talla_cm: -0.304
peso_kg: -0.281
edad_niño2: -0.255
puntuación_índice_riqueza: -0.204
nivel_hemoglobina_ajustada_g/dl_madre: -0.182
amamantamiento: 0.177
región_natural: 0.161
piso_vivienda: -0.139
productos_lácteos: -0.120
n_veces_comida_sólida_semisólida_día_anterior: -0.111
antiparasitarios_últimos_seis_meses: -0.109
área_residencia: 0.106
n_niños_menores_cinco: 0.096
nivel_educativo_esposo_compañero_REC91_22: -0.095
edad_madre: -0.092
lugar_parto: -0.090
diversidad_dietética2: -0.087
hierro_gotas_última_semana: -0.087
niño_afiliado_sis: 0.079
persona_alimenta_niño: -0.078
orden_nacimiento: 0.075
n_veces_lavado_manos: -0.075
durante_embarazo_diagnóstico_anemia: -0.074
nivel_educativo_madre: -0.072


In [ ]:
filtro_nombres_binaria2 = list(filtro_ordenado_binaria2.keys())
filtro_nombres_binaria2

['anemia',
 'talla_cm',
 'peso_kg',
 'edad_niño2',
 'puntuación_índice_riqueza',
 'nivel_hemoglobina_ajustada_g/dl_madre',
 'amamantamiento',
 'región_natural',
 'piso_vivienda',
 'productos_lácteos',
 'n_veces_comida_sólida_semisólida_día_anterior',
 'antiparasitarios_últimos_seis_meses',
 'área_residencia',
 'n_niños_menores_cinco',
 'nivel_educativo_esposo_compañero_REC91_22',
 'edad_madre',
 'lugar_parto',
 'diversidad_dietética2',
 'hierro_gotas_última_semana',
 'niño_afiliado_sis',
 'persona_alimenta_niño',
 'orden_nacimiento',
 'n_veces_lavado_manos',
 'durante_embarazo_diagnóstico_anemia',
 'nivel_educativo_madre']

In [ ]:
df_binaria_filtro2 = df[filtro_nombres_binaria2]
df_binaria_filtro2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9205 entries, 0 to 9204
Data columns (total 25 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   anemia                                         9205 non-null   int64  
 1   talla_cm                                       9205 non-null   float64
 2   peso_kg                                        9205 non-null   float64
 3   edad_niño2                                     9205 non-null   int64  
 4   puntuación_índice_riqueza                      9205 non-null   float64
 5   nivel_hemoglobina_ajustada_g/dl_madre          9205 non-null   float64
 6   amamantamiento                                 9205 non-null   int64  
 7   región_natural                                 9205 non-null   float64
 8   piso_vivienda                                  9205 non-null   int64  
 9   productos_lácteos                              9205 

In [ ]:
y3_filtro = df_binaria_filtro2['anemia']
y3_filtro.shape

(9205,)

In [ ]:
X3_filtro = df_binaria_filtro2.drop('anemia', axis = 1)
X3_filtro.shape

(9205, 24)

<h1 style="color:black;" > 7. Preparación de datos para el modelado <a id="8"></a> </h1>

## Selección de variables

In [ ]:
categoricas_X1 = ['edad_niño2',
               'amamantamiento',
               'región_natural',
               'piso_vivienda',
               'n_veces_comida_sólida_semisólida_día_anterior',
               'orden_nacimiento',
               'nivel_educativo_madre',
               'lugar_parto',
               'nivel_educativo_esposo_compañero_REC91_22',
               'n_veces_lavado_manos',
               'persona_alimenta_niño',
               'fuente_agua',
               'tamaño_niño_al_nacer']
columnas_X1 = X1.columns.tolist()
escalables_X1 = [col for col in columnas_X1 if col not in categoricas_X1]


categoricas_X1_filtro = ['edad_niño2',
               'amamantamiento',
               'región_natural',
               'piso_vivienda',
               'n_veces_comida_sólida_semisólida_día_anterior',
               'orden_nacimiento',
               'nivel_educativo_madre',
               'lugar_parto',
               'nivel_educativo_esposo_compañero_REC91_22',
               'n_veces_lavado_manos',
               'persona_alimenta_niño']
columnas_X1_filtro = X1_filtro.columns.tolist()
escalables_X1_filtro = [col for col in columnas_X1_filtro if col not in categoricas_X1_filtro]


categoricas_X2 = ['edad_niño2',
               'amamantamiento',
               'región_natural',
               'piso_vivienda',
               'n_veces_comida_sólida_semisólida_día_anterior',
               'orden_nacimiento',
               'nivel_educativo_madre',
               'lugar_parto',
               'nivel_educativo_esposo_compañero_REC91_22',
               'n_veces_lavado_manos',
               'persona_alimenta_niño',
               'fuente_agua',
               'tamaño_niño_al_nacer']
columnas_X2 = X2.columns.tolist()
escalables_X2 = [col for col in columnas_X2 if col not in categoricas_X2]

categoricas_X2_filtro = ['edad_niño2',
               'amamantamiento',
               'región_natural',
               'piso_vivienda',
               'n_veces_comida_sólida_semisólida_día_anterior',
               'orden_nacimiento',
               'nivel_educativo_madre',
               'lugar_parto']
columnas_X2_filtro = X2_filtro.columns.tolist()
escalables_X2_filtro = [col for col in columnas_X2_filtro if col not in categoricas_X2_filtro]


categoricas_X3 = ['edad_niño2',
               'amamantamiento',
               'región_natural',
               'piso_vivienda',
               'n_veces_comida_sólida_semisólida_día_anterior',
               'orden_nacimiento',
               'nivel_educativo_madre',
               'lugar_parto',
               'nivel_educativo_esposo_compañero_REC91_22',
               'n_veces_lavado_manos',
               'persona_alimenta_niño',
               'fuente_agua',
               'tamaño_niño_al_nacer']
columnas_X3 = X3.columns.tolist()
escalables_X3 = [col for col in columnas_X3 if col not in categoricas_X3]

categoricas_X3_filtro = ['edad_niño2',
               'amamantamiento',
               'región_natural',
               'piso_vivienda',
               'n_veces_comida_sólida_semisólida_día_anterior',
               'orden_nacimiento',
               'nivel_educativo_madre']
columnas_X3_filtro = X3_filtro.columns.tolist()
escalables_X3_filtro = [col for col in columnas_X3_filtro if col not in categoricas_X3_filtro]

## Escalador

Se escalará usando MinMaxScaler para estandarizar todas las variables en una rango de 0 y 1:

In [ ]:
minmax_scaler = MinMaxScaler()
one_hot_encoder = OneHotEncoder(drop='first')

## Imputador

In [ ]:
imputer_escalables = SimpleImputer(strategy="median")
imputer_categoricas = SimpleImputer(strategy="most_frequent")

## Transformador

In [ ]:
col_transformer_X2 = make_column_transformer((make_pipeline(minmax_scaler, imputer_escalables), escalables_X2),
                                              (make_pipeline(one_hot_encoder, imputer_categoricas), categoricas_X2),
                                              remainder = 'passthrough')
col_transformer_X2


col_transformer_X2_filtro = make_column_transformer((make_pipeline(minmax_scaler, imputer_escalables), escalables_X2_filtro),
                                              (make_pipeline(one_hot_encoder, imputer_categoricas), categoricas_X2_filtro),
                                              remainder = 'passthrough')
col_transformer_X2_filtro


col_transformer_X3 = make_column_transformer((make_pipeline(minmax_scaler, imputer_escalables), escalables_X3),
                                              (make_pipeline(one_hot_encoder, imputer_categoricas), categoricas_X3),
                                              remainder = 'passthrough')
col_transformer_X3


col_transformer_X3_filtro = make_column_transformer((make_pipeline(minmax_scaler, imputer_escalables), escalables_X3_filtro),
                                              (make_pipeline(one_hot_encoder, imputer_categoricas), categoricas_X3_filtro),
                                              remainder = 'passthrough')
col_transformer_X3_filtro

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('minmaxscaler',
                                                  MinMaxScaler()),
                                                 ('simpleimputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['talla_cm', 'peso_kg',
                                  'puntuación_índice_riqueza',
                                  'nivel_hemoglobina_ajustada_g/dl_madre',
                                  'productos_lácteos',
                                  'antiparasitarios_últimos_seis_meses',
                                  'área_residencia', 'n_niños_meno...
                                  'n_veces_lavado_manos',
                                  'durante_embarazo_diagnóstico_anemia']),
                                ('pipeline-2',
                                 Pipeline(steps=[('onehotencoder',
                                                  OneHotEncoder(drop='first')),
                                                 ('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['edad_niño2', 'amamantamiento',
                                  'región_natural', 'piso_vivienda',
                                  'n_veces_comida_sólida_semisólida_día_anterior',
                                  'orden_nacimiento',
                                  'nivel_educativo_madre'])])

## DateFrame de las características transformado

In [ ]:
X2_encoded = pd.DataFrame(col_transformer_X2.fit_transform(X2),
                           columns = col_transformer_X2.get_feature_names_out(),
                           index = X2.index).rename_columns(
    function = lambda x: x.removeprefix('pipeline-1__')).rename_columns(
    function = lambda x: x.removeprefix('pipeline-2__')).rename_columns(
    function = lambda x: x.removeprefix('remainder__'))

X2_encoded.head()

,talla_cm,peso_kg,puntuación_índice_riqueza,nivel_hemoglobina_ajustada_g/dl_madre,productos_lácteos,área_residencia,antiparasitarios_últimos_seis_meses,n_niños_menores_cinco,diversidad_dietética2,hierro_gotas_última_semana,edad_madre,durante_embarazo_diagnóstico_anemia,seguro_salud_essalud/ipss,fiebre_últimas_dos_semanas,n_personas_hogar,res_pollo_pescado_mariscos_otrascarnes_hígado_mondongo_otrasvísceras,diarrea_últimas_dos_semanas,violencia_física_severa,f_plátano_isla_seda_maduro_manzana_palta_otras,imc_madre,harina_pan_fideos_galletas_otra_comida_con_cereales,f_secas,suplemento_vitamina_a_reciente,tos_últimas_dos_semanas,frutas,v_camote_zanahorias_zapallo,huevos,sexo_niños,f_naranjas_mandarina_lima_maracuyá_toronja,preparación_alimentos_minsa_último_año,n_veces_preparación_alimentos_minsa_último_año,comida_hecha_aceite_grasas_mantequilla,habas_frijol_lenteja_soya_pallares_garbanzo_arvejas,papa_yuca_olluco_oca_mashua,actualmente_trabajando,v_hoja_verdeoscuro_espinaca_acelga_otros,b_té_café,f_mango_papaya_aguaje_otrasfrutas_vitamina_a,visita_minsa_alimentación_último_año,n_veces_visita_minsa_alimentación_último_año,actualmente_embarazada,edad_niño2_1,edad_niño2_2,edad_niño2_3,edad_niño2_4,amamantamiento_1,amamantamiento_2,región_natural_2.0,región_natural_3.0,región_natural_4.0,piso_vivienda_21,piso_vivienda_31,piso_vivienda_32,piso_vivienda_33,piso_vivienda_34,n_veces_comida_sólida_semisólida_día_anterior_1,n_veces_comida_sólida_semisólida_día_anterior_2,n_veces_comida_sólida_semisólida_día_anterior_3,n_veces_comida_sólida_semisólida_día_anterior_4,n_veces_comida_sólida_semisólida_día_anterior_5,n_veces_comida_sólida_semisólida_día_anterior_6,n_veces_comida_sólida_semisólida_día_anterior_7,orden_nacimiento_2,orden_nacimiento_3,orden_nacimiento_4,nivel_educativo_madre_1.0,nivel_educativo_madre_2.0,nivel_educativo_madre_3.0,lugar_parto_12.0,lugar_parto_21.0,lugar_parto_22.0,lugar_parto_23.0,lugar_parto_24.0,lugar_parto_25.0,lugar_parto_26.0,lugar_parto_27.0,lugar_parto_31.0,lugar_parto_32.0,lugar_parto_41.0,lugar_parto_42.0,nivel_educativo_esposo_compañero_REC91_22_1,nivel_educativo_esposo_compañero_REC91_22_2,nivel_educativo_esposo_compañero_REC91_22_3,nivel_educativo_esposo_compañero_REC91_22_4,nivel_educativo_esposo_compañero_REC91_22_5,n_veces_lavado_manos_2,n_veces_lavado_manos_3,n_veces_lavado_manos_4,persona_alimenta_niño_2,persona_alimenta_niño_3,persona_alimenta_niño_4,persona_alimenta_niño_5,persona_alimenta_niño_6,persona_alimenta_niño_7,persona_alimenta_niño_8,persona_alimenta_niño_9,fuente_agua_2,fuente_agua_3,fuente_agua_4,fuente_agua_5,tamaño_niño_al_nacer_2.0,tamaño_niño_al_nacer_3.0,tamaño_niño_al_nacer_4.0,tamaño_niño_al_nacer_5.0
0,0.467562,0.504630,0.858686,0.583333,1.0,0.0,1.0,0.0,1.0,1.0,0.205882,1.0,0.0,0.0,0.045455,1.0,0.0,0.0,1.0,0.284515,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.376957,0.337963,0.623869,0.787879,1.0,0.0,0.0,0.0,1.0,1.0,0.558824,1.0,0.0,0.0,0.090909,1.0,0.0,0.0,1.0,0.390467,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.466443,0.356481,0.734347,0.606061,1.0,0.0,1.0,0.0,1.0,1.0,0.500000,1.0,1.0,1.0,0.045455,1.0,0.0,0.0,0.0,0.186466,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0

In [ ]:
X2_filtro_encoded = pd.DataFrame(col_transformer_X2_filtro.fit_transform(X2_filtro),
                           columns = col_transformer_X2_filtro.get_feature_names_out(),
                           index = X2_filtro.index).rename_columns(
    function = lambda x: x.removeprefix('pipeline-1__')).rename_columns(
    function = lambda x: x.removeprefix('pipeline-2__')).rename_columns(
    function = lambda x: x.removeprefix('remainder__'))

X2_filtro_encoded.head()

,talla_cm,peso_kg,puntuación_índice_riqueza,nivel_hemoglobina_ajustada_g/dl_madre,productos_lácteos,área_residencia,antiparasitarios_últimos_seis_meses,n_niños_menores_cinco,diversidad_dietética2,hierro_gotas_última_semana,nivel_educativo_esposo_compañero_REC91_22,edad_madre,durante_embarazo_diagnóstico_anemia,seguro_salud_essalud/ipss,persona_alimenta_niño,n_veces_lavado_manos,fiebre_últimas_dos_semanas,n_personas_hogar,edad_niño2_1,edad_niño2_2,edad_niño2_3,edad_niño2_4,amamantamiento_1,amamantamiento_2,región_natural_2.0,región_natural_3.0,región_natural_4.0,piso_vivienda_21,piso_vivienda_31,piso_vivienda_32,piso_vivienda_33,piso_vivienda_34,n_veces_comida_sólida_semisólida_día_anterior_1,n_veces_comida_sólida_semisólida_día_anterior_2,n_veces_comida_sólida_semisólida_día_anterior_3,n_veces_comida_sólida_semisólida_día_anterior_4,n_veces_comida_sólida_semisólida_día_anterior_5,n_veces_comida_sólida_semisólida_día_anterior_6,n_veces_comida_sólida_semisólida_día_anterior_7,orden_nacimiento_2,orden_nacimiento_3,orden_nacimiento_4,nivel_educativo_madre_1.0,nivel_educativo_madre_2.0,nivel_educativo_madre_3.0,lugar_parto_12.0,lugar_parto_21.0,lugar_parto_22.0,lugar_parto_23.0,lugar_parto_24.0,lugar_parto_25.0,lugar_parto_26.0,lugar_parto_27.0,lugar_parto_31.0,lugar_parto_32.0,lugar_parto_41.0,lugar_parto_42.0
0,0.467562,0.504630,0.858686,0.583333,1.0,0.0,1.0,0.0,1.0,1.0,0.6,0.205882,1.0,0.0,0.0,1.0,0.0,0.045455,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.376957,0.337963,0.623869,0.787879,1.0,0.0,0.0,0.0,1.0,1.0,0.6,0.558824,1.0,0.0,0.0,1.0,0.0,0.090909,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.466443,0.356481,0.734347,0.606061,1.0,0.0,1.0,0.0,1.0,1.0,0.8,0.500000,1.0,1.0,0.0,1.0,1.0,0.045455,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.148770,0.157407,0.559457,0.750000,0.0,0.0,0.0,0.0,1.0,0.0,0.4,0.470588,1.0,0.0,0.0,1.0,0.0,0.136364,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.324385,0.342593,0.736315,0.704545,1.0,0.0,0.0,0.0,1.0,1.0,0.4,0.529412,1.0,1.0,0.0,1.0,0.0,0.090909,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X3_encoded = pd.DataFrame(col_transformer_X3.fit_transform(X3),
                           columns = col_transformer_X3.get_feature_names_out(),
                           index = X3.index).rename_columns(
    function = lambda x: x.removeprefix('pipeline-1__')).rename_columns(
    function = lambda x: x.removeprefix('pipeline-2__')).rename_columns(
    function = lambda x: x.removeprefix('remainder__'))

X3_encoded.head()

,talla_cm,peso_kg,puntuación_índice_riqueza,nivel_hemoglobina_ajustada_g/dl_madre,productos_lácteos,antiparasitarios_últimos_seis_meses,área_residencia,n_niños_menores_cinco,edad_madre,diversidad_dietética2,hierro_gotas_última_semana,niño_afiliado_sis,durante_embarazo_diagnóstico_anemia,fiebre_últimas_dos_semanas,diarrea_últimas_dos_semanas,n_personas_hogar,violencia_física_severa,imc_madre,res_pollo_pescado_mariscos_otrascarnes_hígado_mondongo_otrasvísceras,f_plátano_isla_seda_maduro_manzana_palta_otras,suplemento_vitamina_a_reciente,harina_pan_fideos_galletas_otra_comida_con_cereales,v_camote_zanahorias_zapallo,huevos,sexo_niños,tos_últimas_dos_semanas,f_secas,frutas,f_naranjas_mandarina_lima_maracuyá_toronja,papa_yuca_olluco_oca_mashua,habas_frijol_lenteja_soya_pallares_garbanzo_arvejas,visita_minsa_alimentación_último_año,preparación_alimentos_minsa_último_año,n_veces_preparación_alimentos_minsa_último_año,f_mango_papaya_aguaje_otrasfrutas_vitamina_a,actualmente_embarazada,comida_hecha_aceite_grasas_mantequilla,v_hoja_verdeoscuro_espinaca_acelga_otros,n_veces_visita_minsa_alimentación_último_año,actualmente_trabajando,b_té_café,edad_niño2_1,edad_niño2_2,edad_niño2_3,edad_niño2_4,amamantamiento_1,amamantamiento_2,región_natural_2.0,región_natural_3.0,región_natural_4.0,piso_vivienda_21,piso_vivienda_31,piso_vivienda_32,piso_vivienda_33,piso_vivienda_34,n_veces_comida_sólida_semisólida_día_anterior_1,n_veces_comida_sólida_semisólida_día_anterior_2,n_veces_comida_sólida_semisólida_día_anterior_3,n_veces_comida_sólida_semisólida_día_anterior_4,n_veces_comida_sólida_semisólida_día_anterior_5,n_veces_comida_sólida_semisólida_día_anterior_6,n_veces_comida_sólida_semisólida_día_anterior_7,orden_nacimiento_2,orden_nacimiento_3,orden_nacimiento_4,nivel_educativo_madre_1.0,nivel_educativo_madre_2.0,nivel_educativo_madre_3.0,lugar_parto_12.0,lugar_parto_21.0,lugar_parto_22.0,lugar_parto_23.0,lugar_parto_24.0,lugar_parto_25.0,lugar_parto_26.0,lugar_parto_27.0,lugar_parto_31.0,lugar_parto_32.0,lugar_parto_41.0,lugar_parto_42.0,nivel_educativo_esposo_compañero_REC91_22_1,nivel_educativo_esposo_compañero_REC91_22_2,nivel_educativo_esposo_compañero_REC91_22_3,nivel_educativo_esposo_compañero_REC91_22_4,nivel_educativo_esposo_compañero_REC91_22_5,n_veces_lavado_manos_2,n_veces_lavado_manos_3,n_veces_lavado_manos_4,persona_alimenta_niño_2,persona_alimenta_niño_3,persona_alimenta_niño_4,persona_alimenta_niño_5,persona_alimenta_niño_6,persona_alimenta_niño_7,persona_alimenta_niño_8,persona_alimenta_niño_9,fuente_agua_2,fuente_agua_3,fuente_agua_4,fuente_agua_5,tamaño_niño_al_nacer_2.0,tamaño_niño_al_nacer_3.0,tamaño_niño_al_nacer_4.0,tamaño_niño_al_nacer_5.0
0,0.467562,0.504630,0.858686,0.583333,1.0,1.0,0.0,0.0,0.205882,1.0,1.0,0.0,1.0,0.0,0.0,0.045455,0.0,0.284515,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.376957,0.337963,0.623869,0.787879,1.0,0.0,0.0,0.0,0.558824,1.0,1.0,1.0,1.0,0.0,0.0,0.090909,0.0,0.390467,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.466443,0.356481,0.734347,0.606061,1.0,1.0,0.0,0.0,0.500000,1.0,1.0,0.0,1.0,1.0,0.0,0.045455,0.0,0.186466,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0

In [ ]:
X3_filtro_encoded = pd.DataFrame(col_transformer_X3_filtro.fit_transform(X3_filtro),
                           columns = col_transformer_X3_filtro.get_feature_names_out(),
                           index = X3_filtro.index).rename_columns(
    function = lambda x: x.removeprefix('pipeline-1__')).rename_columns(
    function = lambda x: x.removeprefix('pipeline-2__')).rename_columns(
    function = lambda x: x.removeprefix('remainder__'))

X3_filtro_encoded.head()

,talla_cm,peso_kg,puntuación_índice_riqueza,nivel_hemoglobina_ajustada_g/dl_madre,productos_lácteos,antiparasitarios_últimos_seis_meses,área_residencia,n_niños_menores_cinco,nivel_educativo_esposo_compañero_REC91_22,edad_madre,lugar_parto,diversidad_dietética2,hierro_gotas_última_semana,niño_afiliado_sis,persona_alimenta_niño,n_veces_lavado_manos,durante_embarazo_diagnóstico_anemia,edad_niño2_1,edad_niño2_2,edad_niño2_3,edad_niño2_4,amamantamiento_1,amamantamiento_2,región_natural_2.0,región_natural_3.0,región_natural_4.0,piso_vivienda_21,piso_vivienda_31,piso_vivienda_32,piso_vivienda_33,piso_vivienda_34,n_veces_comida_sólida_semisólida_día_anterior_1,n_veces_comida_sólida_semisólida_día_anterior_2,n_veces_comida_sólida_semisólida_día_anterior_3,n_veces_comida_sólida_semisólida_día_anterior_4,n_veces_comida_sólida_semisólida_día_anterior_5,n_veces_comida_sólida_semisólida_día_anterior_6,n_veces_comida_sólida_semisólida_día_anterior_7,orden_nacimiento_2,orden_nacimiento_3,orden_nacimiento_4,nivel_educativo_madre_1.0,nivel_educativo_madre_2.0,nivel_educativo_madre_3.0
0,0.467562,0.504630,0.858686,0.583333,1.0,1.0,0.0,0.0,0.6,0.205882,0.645161,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.376957,0.337963,0.623869,0.787879,1.0,0.0,0.0,0.0,0.6,0.558824,0.322581,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.466443,0.356481,0.734347,0.606061,1.0,1.0,0.0,0.0,0.8,0.500000,0.354839,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.148770,0.157407,0.559457,0.750000,0.0,0.0,0.0,0.0,0.4,0.470588,0.322581,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.324385,0.342593,0.736315,0.704545,1.0,0.0,0.0,0.0,0.4,0.529412,0.354839,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


<div style="text-align: left; margin-top: 2rem;">
  <button style="padding: 10px 20px; font-size: 12px; cursor: pointer; background-color: #f0f0f0; color: white; border: none;">
    <a href="#indice" style="color: black; text-decoration: none;">Regresar Tabla de contenido</a><br>
  </button>
</div>

## Conjunto de datos de entrenamiento (train) y prueba (test)

In [ ]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2_encoded, y2, test_size=0.3, stratify=y2, random_state=42)
X2_filtro_train, X2_filtro_test, y2_filtro_train, y2_filtro_test = train_test_split(X2_filtro_encoded, y2_filtro, test_size=0.3, stratify=y2, random_state=42)
X3_train, X3_test, y3_train, y3_test = train_test_split(X3_encoded, y3, test_size=0.3, stratify=y3, random_state=42)
X3_filtro_train, X3_filtro_test, y3_filtro_train, y3_filtro_test = train_test_split(X3_filtro_encoded, y3_filtro, test_size=0.3, stratify=y3, random_state=42)

<h1 style="color:black;" > 8. Modelos de aprendizaje automático <a id="21"></a> </h1>

In [ ]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV

# 6.1. Modelo de clasificación multiclase (2)

## 6.1.1. Con todas las características

### A. Validación cruzada

#### Mejor modelo de Regresión Logística

In [ ]:
param_grid_reg_clf_ovr = [{'penalty': ['l1', 'l2'],
                           'C': [0.01, 0.1, 0.5, 1, 2, 5, 10, 50]}]

param_grid_reg_clf_ovo = [{'estimator__penalty': ['l1', 'l2'], 'estimator__C': [0.01, 0.1, 0.5, 1, 2, 5, 10, 50, 100]}]

In [ ]:
grid_reg_clf_ovr = LogisticRegression(solver='liblinear', multi_class='ovr')
grid_reg_clf_ovo = LogisticRegression(solver='liblinear')
clf_ovo = OneVsOneClassifier(grid_reg_clf_ovo)

grid_search_reg_clf_ovr = GridSearchCV(grid_reg_clf_ovr, param_grid_reg_clf_ovr, cv=5, scoring='accuracy')
grid_search_reg_clf_ovo = GridSearchCV(clf_ovo, param_grid_reg_clf_ovo, cv=5, scoring='accuracy')

grid_search_reg_clf_ovr.fit(X2_train, y2_train)
grid_search_reg_clf_ovo.fit(X2_train, y2_train)

GridSearchCV(cv=5,
             estimator=OneVsOneClassifier(estimator=LogisticRegression(solver='liblinear')),
             param_grid=[{'estimator__C': [0.01, 0.1, 0.5, 1, 2, 5, 10, 50,
                                           100],
                          'estimator__penalty': ['l1', 'l2']}],
             scoring='accuracy')

#### Mejor modelo para KNN

In [ ]:
X2_train.shape

(6443, 104)

In [ ]:
param_grid_knn_clf = [{'n_neighbors': list(range(2, 50))}]

In [ ]:
grid_knn_clf = KNeighborsClassifier()

grid_search_knn_clf = GridSearchCV(grid_knn_clf, param_grid_knn_clf, cv=5, scoring='accuracy')

grid_search_knn_clf.fit(X2_train, y2_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15, 16, 17, 18, 19, 20, 21,
                                          22, 23, 24, 25, 26, 27, 28, 29, 30,
                                          31, ...]}],
             scoring='accuracy')

#### Mejor modelo para Árbol de Decisión

In [ ]:
param_grid_tree_clf = [{'max_depth': list(range(1, 70))}]

In [ ]:
grid_tree_clf = DecisionTreeClassifier(random_state=42)

grid_search_tree_clf = GridSearchCV(grid_tree_clf, param_grid_tree_clf, cv=5, scoring='accuracy')

grid_search_tree_clf.fit(X2_train, y2_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                        13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                        23, 24, 25, 26, 27, 28, 29, 30, ...]}],
             scoring='accuracy')

#### Mejor modelo para Bosques Aleatorios

In [ ]:
param_grid_rndf_clf = [{'n_estimators': list(range(100, 701, 100)), 'max_depth': list(range(1, 60))}]

In [ ]:
grid_rndf_clf = RandomForestClassifier()

grid_search_rndf_clf = GridSearchCV(grid_rndf_clf, param_grid_rndf_clf, cv=5, scoring='accuracy')

grid_search_rndf_clf.fit(X2_train, y2_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                        13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                        23, 24, 25, 26, 27, 28, 29, 30, ...],
                          'n_estimators': [100, 200, 300, 400, 500, 600, 700]}],
             scoring='accuracy')

#### Mejor modelo para Aprendizaje Ensamblado

Método de conjunto Bagging:

In [ ]:
param_grid_bag_clf = [{'n_estimators': list(range(100, 701, 100)),
                      'max_samples': list(range(500, 1501, 100))}]

In [ ]:
grid_bag_clf = BaggingClassifier(DecisionTreeClassifier(), bootstrap=True, random_state=42)

grid_search_bag_clf = GridSearchCV(grid_bag_clf, param_grid_bag_clf, cv=5, scoring='accuracy')

grid_search_bag_clf.fit(X2_train, y2_train)

GridSearchCV(cv=5,
             estimator=BaggingClassifier(estimator=DecisionTreeClassifier(),
                                         random_state=42),
             param_grid=[{'max_samples': [500, 600, 700, 800, 900, 1000, 1100,
                                          1200, 1300, 1400, 1500],
                          'n_estimators': [100, 200, 300, 400, 500, 600, 700]}],
             scoring='accuracy')

Método de conjunto Pasting:

In [ ]:
param_grid_pas_clf = [{'n_estimators': list(range(100, 701, 100)),
                       'max_samples': list(range(500, 1501, 100))}]

In [ ]:
grid_pas_clf = BaggingClassifier(DecisionTreeClassifier(), bootstrap=False, random_state=42)

grid_search_pas_clf = GridSearchCV(grid_pas_clf, param_grid_pas_clf, cv=5, scoring='accuracy')

grid_search_pas_clf.fit(X2_train, y2_train)

GridSearchCV(cv=5,
             estimator=BaggingClassifier(bootstrap=False,
                                         estimator=DecisionTreeClassifier(),
                                         random_state=42),
             param_grid=[{'max_samples': [500, 600, 700, 800, 900, 1000, 1100,
                                          1200, 1300, 1400, 1500],
                          'n_estimators': [100, 200, 300, 400, 500, 600, 700]}],
             scoring='accuracy')

### B. Mejores hiperparámetros, puntajes y estimadores

In [ ]:
best_params_reg_clf_ovr = grid_search_reg_clf_ovr.best_params_
best_params_reg_clf_ovo = grid_search_reg_clf_ovo.best_params_
best_params_knn_clf = grid_search_knn_clf.best_params_
best_params_tree_clf = grid_search_tree_clf.best_params_
best_params_rndf_clf = grid_search_rndf_clf.best_params_
best_params_bag_clf = grid_search_bag_clf.best_params_
best_params_pas_clf = grid_search_pas_clf.best_params_

In [ ]:
best_score_reg_clf_ovr = grid_search_reg_clf_ovr.best_score_
best_score_reg_clf_ovo = grid_search_reg_clf_ovo.best_score_
best_score_knn_clf = grid_search_knn_clf.best_score_
best_score_tree_clf = grid_search_tree_clf.best_score_
best_score_rndf_clf = grid_search_rndf_clf.best_score_
best_score_bag_clf = grid_search_bag_clf.best_score_
best_score_pas_clf = grid_search_pas_clf.best_score_

In [ ]:
best_estimator_reg_clf_ovr = grid_search_reg_clf_ovr.best_estimator_
best_estimator_reg_clf_ovo = grid_search_reg_clf_ovo.best_estimator_
best_estimator_knn_clf = grid_search_knn_clf.best_estimator_
best_estimator_tree_clf = grid_search_tree_clf.best_estimator_
best_estimator_rndf_clf = grid_search_rndf_clf.best_estimator_
best_estimator_bag_clf = grid_search_bag_clf.best_estimator_
best_estimator_pas_clf = grid_search_pas_clf.best_estimator_

### C. Evaluación de los modelos

In [ ]:
best_params = {
    'Regresión Logística OvR': best_params_reg_clf_ovr,
    'Regresión Logística OvO': best_params_reg_clf_ovo,
    'KNN': best_params_knn_clf,
    'Árbol de Decisión': best_params_tree_clf,
    'Bosques Aleatorios': best_params_rndf_clf,
    'Bagging': best_params_bag_clf,
    'Pasting': best_params_pas_clf
}

best_scores = {
    'Regresión Logística OvR': best_score_reg_clf_ovr,
    'Regresión Logística OvO': best_score_reg_clf_ovo,
    'KNN': best_score_knn_clf,
    'Árbol de Decisión': best_score_tree_clf,
    'Bosques Aleatorios': best_score_rndf_clf,
    'Bagging': best_score_bag_clf,
    'Pasting': best_score_pas_clf
}

best_estimators = {
    'Regresión Logística OvR': best_estimator_reg_clf_ovr,
    'Regresión Logística OvO': best_estimator_reg_clf_ovo,
    'KNN': best_estimator_knn_clf,
    'Árbol de Decisión': best_estimator_tree_clf,
    'Bosques Aleatorios': best_estimator_rndf_clf,
    'Bagging': best_estimator_bag_clf,
    'Pasting': best_estimator_pas_clf
}

y_pred = {
    'Regresión Logística OvR': best_estimator_reg_clf_ovr.predict(X2_test),
    'Regresión Logística OvO': best_estimator_reg_clf_ovo.predict(X2_test),
    'KNN': best_estimator_knn_clf.predict(X2_test),
    'Árbol de Decisión': best_estimator_tree_clf.predict(X2_test),
    'Bosques Aleatorios': best_estimator_rndf_clf.predict(X2_test),
    'Bagging': best_estimator_bag_clf.predict(X2_test),
    'Pasting': best_estimator_pas_clf.predict(X2_test)
}

precision_scores = {}
recall_scores = {}
f1_scores = {}

for model, y_pred_test in y_pred.items():
    precision, recall, f1, _ = precision_recall_fscore_support(y2_test, y_pred_test, average=None, zero_division=1)
    precision_scores[model] = precision
    recall_scores[model] = recall
    f1_scores[model] = f1

sobreajuste = {}
for model, estimator in best_estimators.items():
    score_train = estimator.score(X2_train, y2_train)
    score_test = estimator.score(X2_test, y2_test)
    sobreajuste[model] = abs(score_train - score_test)

metricas_generales = pd.DataFrame({
    'Modelos': list(best_params.keys()),
    'Mejores Parámetros': list(best_params.values()),
    'Mejor Score (Train)': list(best_scores.values()),
    'Accuracy (Test)': [accuracy_score(y2_test, y_pred[model]) for model in best_params.keys()],
    'Sobreajuste': list(sobreajuste.values())
})

metricas_generales.set_index('Modelos', inplace=True)

metricas_clases = []
classes = len(set(y2_test))

for i in range(classes):
    metricas_clase = pd.DataFrame({
        'Modelos': list(best_params.keys()),
        f'Precision (Test) Clase {i}': [precision[i] for precision in precision_scores.values()],
        f'Recall (Test) Clase {i}': [recall[i] for recall in recall_scores.values()],
        f'F1 Score (Test) Clase {i}': [f1[i] for f1 in f1_scores.values()]
    })
    metricas_clase.set_index('Modelos', inplace=True)
    metricas_clases.append(metricas_clase)

def get_metricas_detalladas_por_clase(clase):
    if clase < 0 or clase >= classes:
        raise ValueError("Número de clase inválido.")
    return metricas_clases[clase]

In [ ]:
metricas_generales

,Mejores Parámetros,Mejor Score (Train),Accuracy (Test),Sobreajuste
Modelos,,,,
Regresión Logística OvR,"{'C': 2, 'penalty': 'l2'}",0.589479,0.573860,0.030207
Regresión Logística OvO,"{'estimator__C': 5, 'estimator__penalty': 'l2'}",0.594755,0.579290,0.031916
KNN,{'n_neighbors': 35},0.570699,0.564808,0.033826
Árbol de Decisión,{'max_depth': 11},0.604066,0.616220,0.157177
Bosques Aleatorios,"{'max_depth': 41, 'n_estimators': 400}",0.662737,0.664736,0.335264
Bagging,"{'max_samples': 1100, 'n_estimators': 700}",0.656527,0.657495,0.164328
Pasting,"{'max_samples': 1500, 'n_estimators': 600}",0.658545,0.661115,0.226670


In [ ]:
clase = 0
metricas_clase_especifica = get_metricas_detalladas_por_clase(clase)
metricas_clase_especifica

,Precision (Test) Clase 0,Recall (Test) Clase 0,F1 Score (Test) Clase 0
Modelos,,,
Regresión Logística OvR,1.0,0.0,0.0
Regresión Logística OvO,0.0,0.0,0.0
KNN,1.0,0.0,0.0
Árbol de Decisión,0.0,0.0,0.0
Bosques Aleatorios,1.0,0.0,0.0
Bagging,1.0,0.0,0.0
Pasting,1.0,0.0,0.0


In [ ]:
clase = 1
metricas_clase_especifica = get_metricas_detalladas_por_clase(clase)
metricas_clase_especifica

,Precision (Test) Clase 1,Recall (Test) Clase 1,F1 Score (Test) Clase 1
Modelos,,,
Regresión Logística OvR,0.465035,0.292952,0.359459
Regresión Logística OvO,0.489489,0.359031,0.414231
KNN,0.513966,0.202643,0.290679
Árbol de Decisión,0.522936,0.376652,0.437900
Bosques Aleatorios,0.649635,0.392070,0.489011
Bagging,0.603390,0.392070,0.475300
Pasting,0.610932,0.418502,0.496732


In [ ]:
clase = 2
metricas_clase_especifica = get_metricas_detalladas_por_clase(clase)
metricas_clase_especifica

,Precision (Test) Clase 2,Recall (Test) Clase 2,F1 Score (Test) Clase 2
Modelos,,,
Regresión Logística OvR,0.409563,0.243210,0.305190
Regresión Logística OvO,0.421569,0.265432,0.325758
KNN,0.399240,0.259259,0.314371
Árbol de Decisión,0.494444,0.439506,0.465359
Bosques Aleatorios,0.629630,0.356790,0.455477
Bagging,0.600000,0.381481,0.466415
Pasting,0.601179,0.377778,0.463988


In [ ]:
clase = 3
metricas_clase_especifica = get_metricas_detalladas_por_clase(clase)
metricas_clase_especifica

,Precision (Test) Clase 3,Recall (Test) Clase 3,F1 Score (Test) Clase 3
Modelos,,,
Regresión Logística OvR,0.629073,0.840589,0.719610
Regresión Logística OvO,0.637454,0.818486,0.716716
KNN,0.611570,0.842599,0.708732
Árbol de Decisión,0.685931,0.787006,0.733001
Bosques Aleatorios,0.674717,0.916946,0.777399
Bagging,0.680840,0.890154,0.771553
Pasting,0.684861,0.890824,0.774381


## 6.1.2. Con las características más relacionadas

### A. Validación cruzada

#### Mejor modelo de Regresión Logística

In [ ]:
param_grid_reg_clf_ovr_filtro = [{'penalty': ['l1', 'l2'],
                           'C': [0.01, 0.1, 0.5, 1, 2, 5, 10, 50]}]

param_grid_reg_clf_ovo_filtro = [{'estimator__penalty': ['l1', 'l2'], 'estimator__C': [0.01, 0.1, 0.5, 1, 2, 5, 10, 50]}]

In [ ]:
grid_reg_clf_ovr_filtro = LogisticRegression(solver='liblinear', multi_class='ovr')
grid_reg_clf_ovo_filtro = LogisticRegression(solver='liblinear')
clf_ovo_filtro = OneVsOneClassifier(grid_reg_clf_ovo_filtro)

grid_search_reg_clf_ovr_filtro = GridSearchCV(grid_reg_clf_ovr_filtro, param_grid_reg_clf_ovr_filtro, cv=StratifiedKFold(n_splits=5), scoring='accuracy')
grid_search_reg_clf_ovo_filtro = GridSearchCV(clf_ovo_filtro, param_grid_reg_clf_ovo_filtro, cv=StratifiedKFold(n_splits=5), scoring='accuracy')

grid_search_reg_clf_ovr_filtro.fit(X2_filtro_train, y2_filtro_train)
grid_search_reg_clf_ovo_filtro.fit(X2_filtro_train, y2_filtro_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=OneVsOneClassifier(estimator=LogisticRegression(solver='liblinear')),
             param_grid=[{'estimator__C': [0.01, 0.1, 0.5, 1, 2, 5, 10, 50],
                          'estimator__penalty': ['l1', 'l2']}],
             scoring='accuracy')

#### Mejor modelo para KNN

In [ ]:
X2_filtro_train.shape

(6443, 57)

In [ ]:
param_grid_knn_clf_filtro = [{'n_neighbors': list(range(2, 57))}]

In [ ]:
grid_knn_clf_filtro = KNeighborsClassifier()

grid_search_knn_clf_filtro = GridSearchCV(grid_knn_clf_filtro, param_grid_knn_clf_filtro, cv=StratifiedKFold(n_splits=5), scoring='accuracy')

grid_search_knn_clf_filtro.fit(X2_filtro_train, y2_filtro_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15, 16, 17, 18, 19, 20, 21,
                                          22, 23, 24, 25, 26, 27, 28, 29, 30,
                                          31, ...]}],
             scoring='accuracy')

#### Mejor modelo para Árbol de Decisión

In [ ]:
param_grid_tree_clf_filtro = [{'max_depth': list(range(1, 57))}]

In [ ]:
grid_tree_clf_filtro = DecisionTreeClassifier(random_state=42)

grid_search_tree_clf_filtro = GridSearchCV(grid_tree_clf_filtro, param_grid_tree_clf_filtro, cv=StratifiedKFold(n_splits=5), scoring='accuracy')

grid_search_tree_clf_filtro.fit(X2_filtro_train, y2_filtro_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=DecisionTreeClassifier(random_state=42),
             param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                        13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                        23, 24, 25, 26, 27, 28, 29, 30, ...]}],
             scoring='accuracy')

#### Mejor modelo para Bosques Aleatorios

In [ ]:
param_grid_rndf_clf_filtro = [{'n_estimators': list(range(100, 401, 100)), 'max_depth': list(range(1, 57))}]

In [ ]:
grid_rndf_clf_filtro = RandomForestClassifier()

grid_search_rndf_clf_filtro = GridSearchCV(grid_rndf_clf_filtro, param_grid_rndf_clf_filtro, cv=StratifiedKFold(n_splits=5), scoring='accuracy')

grid_search_rndf_clf_filtro.fit(X2_filtro_train, y2_filtro_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(),
             param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                        13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                        23, 24, 25, 26, 27, 28, 29, 30, ...],
                          'n_estimators': [100, 200, 300, 400]}],
             scoring='accuracy')

#### Mejor modelo para Aprendizaje Ensamblado

Método de conjunto Bagging:

In [ ]:
param_grid_bag_clf_filtro = [{'n_estimators': list(range(100, 401, 100)),
                      'max_samples': list(range(500, 1001, 100))}]

In [ ]:
grid_bag_clf_filtro = BaggingClassifier(DecisionTreeClassifier(), bootstrap=True, random_state=42)

grid_search_bag_clf_filtro = GridSearchCV(grid_bag_clf_filtro, param_grid_bag_clf_filtro, cv=StratifiedKFold(n_splits=5), scoring='accuracy')

grid_search_bag_clf_filtro.fit(X2_filtro_train, y2_filtro_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=BaggingClassifier(estimator=DecisionTreeClassifier(),
                                         random_state=42),
             param_grid=[{'max_samples': [500, 600, 700, 800, 900, 1000],
                          'n_estimators': [100, 200, 300, 400]}],
             scoring='accuracy')

Método de conjunto Pasting:

In [ ]:
param_grid_pas_clf_filtro = [{'n_estimators': list(range(100, 401, 100)),
                       'max_samples': list(range(500, 1001, 100))}]

In [ ]:
grid_pas_clf_filtro = BaggingClassifier(DecisionTreeClassifier(), bootstrap=False, random_state=42)

grid_search_pas_clf_filtro = GridSearchCV(grid_pas_clf_filtro, param_grid_pas_clf_filtro, cv=StratifiedKFold(n_splits=5), scoring='accuracy')

grid_search_pas_clf_filtro.fit(X2_filtro_train, y2_filtro_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=BaggingClassifier(bootstrap=False,
                                         estimator=DecisionTreeClassifier(),
                                         random_state=42),
             param_grid=[{'max_samples': [500, 600, 700, 800, 900, 1000],
                          'n_estimators': [100, 200, 300, 400]}],
             scoring='accuracy')

### B. Mejores hiperparámetros, puntajes y estimadores

In [ ]:
best_params_reg_clf_ovr_filtro = grid_search_reg_clf_ovr_filtro.best_params_
best_params_reg_clf_ovo_filtro = grid_search_reg_clf_ovo_filtro.best_params_
best_params_knn_clf_filtro = grid_search_knn_clf_filtro.best_params_
best_params_tree_clf_filtro = grid_search_tree_clf_filtro.best_params_
best_params_rndf_clf_filtro = grid_search_rndf_clf_filtro.best_params_
best_params_bag_clf_filtro = grid_search_bag_clf_filtro.best_params_
best_params_pas_clf_filtro = grid_search_pas_clf_filtro.best_params_

In [ ]:
best_score_reg_clf_ovr_filtro = grid_search_reg_clf_ovr_filtro.best_score_
best_score_reg_clf_ovo_filtro = grid_search_reg_clf_ovo_filtro.best_score_
best_score_knn_clf_filtro = grid_search_knn_clf_filtro.best_score_
best_score_tree_clf_filtro = grid_search_tree_clf_filtro.best_score_
best_score_rndf_clf_filtro = grid_search_rndf_clf_filtro.best_score_
best_score_bag_clf_filtro = grid_search_bag_clf_filtro.best_score_
best_score_pas_clf_filtro = grid_search_pas_clf_filtro.best_score_

In [ ]:
best_estimator_reg_clf_ovr_filtro = grid_search_reg_clf_ovr_filtro.best_estimator_
best_estimator_reg_clf_ovo_filtro = grid_search_reg_clf_ovo_filtro.best_estimator_
best_estimator_knn_clf_filtro = grid_search_knn_clf_filtro.best_estimator_
best_estimator_tree_clf_filtro = grid_search_tree_clf_filtro.best_estimator_
best_estimator_rndf_clf_filtro = grid_search_rndf_clf_filtro.best_estimator_
best_estimator_bag_clf_filtro = grid_search_bag_clf_filtro.best_estimator_
best_estimator_pas_clf_filtro = grid_search_pas_clf_filtro.best_estimator_

### C. Evaluación de los modelos

In [ ]:
best_params_filtro = {
    'Regresión Logística OvR': best_params_reg_clf_ovr_filtro,
    'Regresión Logística OvO': best_params_reg_clf_ovo_filtro,
    'KNN': best_params_knn_clf_filtro,
    'Árbol de Decisión': best_params_tree_clf_filtro,
    'Bosques Aleatorios': best_params_rndf_clf_filtro,
    'Bagging': best_params_bag_clf_filtro,
    'Pasting': best_params_pas_clf_filtro
}

best_scores_filtro = {
    'Regresión Logística OvR': best_score_reg_clf_ovr_filtro,
    'Regresión Logística OvO': best_score_reg_clf_ovo_filtro,
    'KNN': best_score_knn_clf_filtro,
    'Árbol de Decisión': best_score_tree_clf_filtro,
    'Bosques Aleatorios': best_score_rndf_clf_filtro,
    'Bagging': best_score_bag_clf_filtro,
    'Pasting': best_score_pas_clf_filtro
}

best_estimators_filtro = {
    'Regresión Logística OvR': best_estimator_reg_clf_ovr_filtro,
    'Regresión Logística OvO': best_estimator_reg_clf_ovo_filtro,
    'KNN': best_estimator_knn_clf_filtro,
    'Árbol de Decisión': best_estimator_tree_clf_filtro,
    'Bosques Aleatorios': best_estimator_rndf_clf_filtro,
    'Bagging': best_estimator_bag_clf_filtro,
    'Pasting': best_estimator_pas_clf_filtro
}

y_pred_filtro = {
    'Regresión Logística OvR': best_estimator_reg_clf_ovr_filtro.predict(X2_filtro_test),
    'Regresión Logística OvO': best_estimator_reg_clf_ovo_filtro.predict(X2_filtro_test),
    'KNN': best_estimator_knn_clf_filtro.predict(X2_filtro_test),
    'Árbol de Decisión': best_estimator_tree_clf_filtro.predict(X2_filtro_test),
    'Bosques Aleatorios': best_estimator_rndf_clf_filtro.predict(X2_filtro_test),
    'Bagging': best_estimator_bag_clf_filtro.predict(X2_filtro_test),
    'Pasting': best_estimator_pas_clf_filtro.predict(X2_filtro_test)
}

precision_scores_filtro = {}
recall_scores_filtro = {}
f1_scores_filtro = {}

for model, y_pred_test in y_pred_filtro.items():
    precision, recall, f1, _ = precision_recall_fscore_support(y2_filtro_test, y_pred_test, average=None, zero_division=1)
    precision_scores_filtro[model] = precision
    recall_scores_filtro[model] = recall
    f1_scores_filtro[model] = f1

sobreajuste_filtro = {}
for model, estimator in best_estimators_filtro.items():
    score_train = estimator.score(X2_filtro_train, y2_filtro_train)
    score_test = estimator.score(X2_filtro_test, y2_filtro_test)
    sobreajuste_filtro[model] = abs(score_train - score_test)

metricas_generales_filtro = pd.DataFrame({
    'Modelos': list(best_params_filtro.keys()),
    'Mejores Parámetros': list(best_params_filtro.values()),
    'Mejor Score (Train)': list(best_scores_filtro.values()),
    'Accuracy (Test)': [accuracy_score(y2_filtro_test, y_pred_filtro[model]) for model in best_params_filtro.keys()],
    'Sobreajuste': list(sobreajuste_filtro.values())
})

metricas_generales_filtro.set_index('Modelos', inplace=True)

metricas_clases_filtro = []
classes = len(set(y2_test))

for i in range(classes):
    metricas_clase_filtro = pd.DataFrame({
        'Modelos': list(best_params_filtro.keys()),
        f'Precision (Test) Clase {i}': [precision[i] for precision in precision_scores_filtro.values()],
        f'Recall (Test) Clase {i}': [recall[i] for recall in recall_scores_filtro.values()],
        f'F1 Score (Test) Clase {i}': [f1[i] for f1 in f1_scores_filtro.values()]
    })
    metricas_clase_filtro.set_index('Modelos', inplace=True)
    metricas_clases_filtro.append(metricas_clase_filtro)

def get_metricas_detalladas_por_clase_filtro(clase):
    if clase < 0 or clase >= classes:
        raise ValueError("Número de clase inválido.")
    return metricas_clases_filtro[clase]

In [ ]:
metricas_generales_filtro

,Mejores Parámetros,Mejor Score (Train),Accuracy (Test),Sobreajuste
Modelos,,,,
Regresión Logística OvR,"{'C': 0.5, 'penalty': 'l1'}",0.582494,0.571325,0.017221
Regresión Logística OvO,"{'estimator__C': 10, 'estimator__penalty': 'l1'}",0.584821,0.573860,0.020274
KNN,{'n_neighbors': 46},0.573956,0.557205,0.042826
Árbol de Decisión,{'max_depth': 9},0.609965,0.607169,0.090953
Bosques Aleatorios,"{'max_depth': 17, 'n_estimators': 300}",0.661030,0.665460,0.333143
Bagging,"{'max_samples': 1000, 'n_estimators': 200}",0.651561,0.661115,0.140685
Pasting,"{'max_samples': 900, 'n_estimators': 400}",0.655130,0.658219,0.139081


In [ ]:
clase = 0
metricas_clase_especifica = get_metricas_detalladas_por_clase_filtro(clase)
metricas_clase_especifica

,Precision (Test) Clase 0,Recall (Test) Clase 0,F1 Score (Test) Clase 0
Modelos,,,
Regresión Logística OvR,1.0,0.0,0.0
Regresión Logística OvO,0.0,0.0,0.0
KNN,1.0,0.0,0.0
Árbol de Decisión,0.0,0.0,0.0
Bosques Aleatorios,1.0,0.0,0.0
Bagging,1.0,0.0,0.0
Pasting,1.0,0.0,0.0


In [ ]:
clase = 1
metricas_clase_especifica = get_metricas_detalladas_por_clase_filtro(clase)
metricas_clase_especifica

,Precision (Test) Clase 1,Recall (Test) Clase 1,F1 Score (Test) Clase 1
Modelos,,,
Regresión Logística OvR,0.416370,0.257709,0.318367
Regresión Logística OvO,0.421538,0.301762,0.351733
KNN,0.452586,0.231278,0.306122
Árbol de Decisión,0.581395,0.330396,0.421348
Bosques Aleatorios,0.640678,0.416300,0.504673
Bagging,0.609272,0.405286,0.486772
Pasting,0.602606,0.407489,0.486202


In [ ]:
clase = 2
metricas_clase_especifica = get_metricas_detalladas_por_clase_filtro(clase)
metricas_clase_especifica

,Precision (Test) Clase 2,Recall (Test) Clase 2,F1 Score (Test) Clase 2
Modelos,,,
Regresión Logística OvR,0.404313,0.185185,0.254022
Regresión Logística OvO,0.411504,0.229630,0.294770
KNN,0.392786,0.241975,0.299465
Árbol de Decisión,0.462751,0.398765,0.428382
Bosques Aleatorios,0.628392,0.371605,0.467029
Bagging,0.601103,0.403704,0.483013
Pasting,0.606654,0.382716,0.469341


In [ ]:
clase = 3
metricas_clase_especifica = get_metricas_detalladas_por_clase_filtro(clase)
metricas_clase_especifica

,Precision (Test) Clase 3,Recall (Test) Clase 3,F1 Score (Test) Clase 3
Modelos,,,
Regresión Logística OvR,0.621327,0.878098,0.727727
Regresión Logística OvO,0.636089,0.845278,0.725913
KNN,0.609552,0.829203,0.702611
Árbol de Decisión,0.667036,0.806430,0.730139
Bosques Aleatorios,0.678068,0.902880,0.774490
Bagging,0.686326,0.880777,0.771487
Pasting,0.680556,0.886135,0.769857


# 6.2. Modelo de clasificación binomial (3)

## 6.2.1. Con todas las características

### A. Validación cruzada

#### Mejor modelo de Regresión Logística

In [ ]:
param_grid_reg_clf_bi = [{'penalty': ['l1', 'l2'],
                           'C': [0.01, 0.1, 1.0, 10, 100]}]

In [ ]:
grid_reg_clf_bi = LogisticRegression(solver='liblinear')

grid_search_reg_clf_bi = GridSearchCV(grid_reg_clf_bi, param_grid_reg_clf_bi, cv=5, scoring='accuracy')

grid_search_reg_clf_bi.fit(X3_train, y3_train)

GridSearchCV(cv=5, estimator=LogisticRegression(solver='liblinear'),
             param_grid=[{'C': [0.01, 0.1, 1.0, 10, 100],
                          'penalty': ['l1', 'l2']}],
             scoring='accuracy')

#### Mejor modelo para KNN

In [ ]:
X3_train.shape

(6443, 104)

In [ ]:
param_grid_knn_clf_bi = [{'n_neighbors': list(range(2, 104))}]

In [ ]:
grid_knn_clf_bi = KNeighborsClassifier()

grid_search_knn_clf_bi = GridSearchCV(grid_knn_clf_bi, param_grid_knn_clf_bi, cv=5, scoring='accuracy')

grid_search_knn_clf_bi.fit(X3_train, y3_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15, 16, 17, 18, 19, 20, 21,
                                          22, 23, 24, 25, 26, 27, 28, 29, 30,
                                          31, ...]}],
             scoring='accuracy')

#### Mejor modelo para Árbol de Decisión

In [ ]:
param_grid_tree_clf_bi = [{'max_depth': list(range(1, 104))}]

In [ ]:
grid_tree_clf_bi = DecisionTreeClassifier(random_state=42)

grid_search_tree_clf_bi = GridSearchCV(grid_tree_clf_bi, param_grid_tree_clf_bi, cv=5, scoring='accuracy')

grid_search_tree_clf_bi.fit(X3_train, y3_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                        13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                        23, 24, 25, 26, 27, 28, 29, 30, ...]}],
             scoring='accuracy')

#### Mejor modelo para Bosques Aleatorios

In [ ]:
param_grid_rndf_clf_bi = [{'n_estimators': list(range(100, 301, 100))},
                       {'n_estimators': list(range(100, 301, 100)), 'max_depth': list(range(1, 104))}]

In [ ]:
grid_rndf_clf_bi = RandomForestClassifier()

grid_search_rndf_clf_bi = GridSearchCV(grid_rndf_clf_bi, param_grid_rndf_clf_bi, cv=5, scoring='accuracy')

grid_search_rndf_clf_bi.fit(X3_train, y3_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid=[{'n_estimators': [100, 200, 300]},
                         {'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                        13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                        23, 24, 25, 26, 27, 28, 29, 30, ...],
                          'n_estimators': [100, 200, 300]}],
             scoring='accuracy')

#### Mejor modelo para Aprendizaje Ensamblado

Método de conjunto Bagging:

In [ ]:
param_grid_bag_clf_bi = [{'n_estimators': list(range(50, 101, 50)),
                      'max_samples': list(range(500, 901, 200))}]

In [ ]:
grid_bag_clf_bi = BaggingClassifier(DecisionTreeClassifier(), bootstrap=True, random_state=42)

grid_search_bag_clf_bi = GridSearchCV(grid_bag_clf_bi, param_grid_bag_clf_bi, cv=5, scoring='accuracy')

grid_search_bag_clf_bi.fit(X3_train, y3_train)

GridSearchCV(cv=5,
             estimator=BaggingClassifier(estimator=DecisionTreeClassifier(),
                                         random_state=42),
             param_grid=[{'max_samples': [500, 700, 900],
                          'n_estimators': [50, 100]}],
             scoring='accuracy')

Método de conjunto Pasting:

In [ ]:
param_grid_pas_clf_bi = [{'n_estimators': list(range(50, 101, 50)),
                       'max_samples': list(range(500, 901, 200))}]

In [ ]:
grid_pas_clf_bi = BaggingClassifier(DecisionTreeClassifier(), bootstrap=False, random_state=42)

grid_search_pas_clf_bi = GridSearchCV(grid_pas_clf_bi, param_grid_pas_clf_bi, cv=5, scoring='accuracy')

grid_search_pas_clf_bi.fit(X3_train, y3_train)

GridSearchCV(cv=5,
             estimator=BaggingClassifier(bootstrap=False,
                                         estimator=DecisionTreeClassifier(),
                                         random_state=42),
             param_grid=[{'max_samples': [500, 700, 900],
                          'n_estimators': [50, 100]}],
             scoring='accuracy')

### B. Mejores hiperparámetros, puntajes y estimadores

In [ ]:
best_params_reg_clf_bi = grid_search_reg_clf_bi.best_params_
best_params_knn_clf_bi = grid_search_knn_clf_bi.best_params_
best_params_tree_clf_bi = grid_search_tree_clf_bi.best_params_
best_params_rndf_clf_bi = grid_search_rndf_clf_bi.best_params_
best_params_bag_clf_bi = grid_search_bag_clf_bi.best_params_
best_params_pas_clf_bi = grid_search_pas_clf_bi.best_params_

In [ ]:
best_score_reg_clf_bi = grid_search_reg_clf_bi.best_score_
best_score_knn_clf_bi = grid_search_knn_clf_bi.best_score_
best_score_tree_clf_bi = grid_search_tree_clf_bi.best_score_
best_score_rndf_clf_bi = grid_search_rndf_clf_bi.best_score_
best_score_bag_clf_bi = grid_search_bag_clf_bi.best_score_
best_score_pas_clf_bi = grid_search_pas_clf_bi.best_score_

In [ ]:
best_estimator_reg_clf_bi = grid_search_reg_clf_bi.best_estimator_
best_estimator_knn_clf_bi = grid_search_knn_clf_bi.best_estimator_
best_estimator_tree_clf_bi = grid_search_tree_clf_bi.best_estimator_
best_estimator_rndf_clf_bi = grid_search_rndf_clf_bi.best_estimator_
best_estimator_bag_clf_bi = grid_search_bag_clf_bi.best_estimator_
best_estimator_pas_clf_bi = grid_search_pas_clf_bi.best_estimator_

### C. Evaluación de los modelos

In [ ]:
best_params = {
    'Regresión Logística': best_params_reg_clf_bi,
    'KNN': best_params_knn_clf_bi,
    'Árbol de Decisión': best_params_tree_clf_bi,
    'Bosques Aleatorios': best_params_rndf_clf_bi,
    'Bagging': best_params_bag_clf_bi,
    'Pasting': best_params_pas_clf_bi
}

best_scores = {
    'Regresión Logística': best_score_reg_clf_bi,
    'KNN': best_score_knn_clf_bi,
    'Árbol de Decisión': best_score_tree_clf_bi,
    'Bosques Aleatorios': best_score_rndf_clf_bi,
    'Bagging': best_score_bag_clf_bi,
    'Pasting': best_score_pas_clf_bi
}

best_estimators = {
    'Regresión Logística': best_estimator_reg_clf_bi,
    'KNN': best_estimator_knn_clf_bi,
    'Árbol de Decisión': best_estimator_tree_clf_bi,
    'Bosques Aleatorios': best_estimator_rndf_clf_bi,
    'Bagging': best_estimator_bag_clf_bi,
    'Pasting': best_estimator_pas_clf_bi
}

y_pred = {
    'Regresión Logística': best_estimator_reg_clf_bi.predict(X3_test),
    'KNN': best_estimator_knn_clf_bi.predict(X3_test),
    'Árbol de Decisión': best_estimator_tree_clf_bi.predict(X3_test),
    'Bosques Aleatorios': best_estimator_rndf_clf_bi.predict(X3_test),
    'Bagging': best_estimator_bag_clf_bi.predict(X3_test),
    'Pasting': best_estimator_pas_clf_bi.predict(X3_test)
}

precision_scores = {}
recall_scores = {}
f1_scores = {}

for model, y_pred_test in y_pred.items():
    precision, recall, f1, _ = precision_recall_fscore_support(y3_test, y_pred_test, average=None, zero_division=1)
    precision_scores[model] = precision
    recall_scores[model] = recall
    f1_scores[model] = f1

sobreajuste = {}
for model, estimator in best_estimators.items():
    score_train = estimator.score(X3_train, y3_train)
    score_test = estimator.score(X3_test, y3_test)
    sobreajuste[model] = abs(score_train - score_test)

metricas_generales = pd.DataFrame({
    'Modelos': list(best_params.keys()),
    'Mejores Parámetros': list(best_params.values()),
    'Mejor Score (Train)': list(best_scores.values()),
    'Accuracy (Test)': [accuracy_score(y3_test, y_pred[model]) for model in best_params.keys()],
    'Sobreajuste': list(sobreajuste.values())
})

metricas_generales.set_index('Modelos', inplace=True)

metricas_clases = []
classes = len(set(y3_test))

for i in range(classes):
    metricas_clase = pd.DataFrame({
        'Modelos': list(best_params.keys()),
        f'Precision (Test) Clase {i}': [precision[i] for precision in precision_scores.values()],
        f'Recall (Test) Clase {i}': [recall[i] for recall in recall_scores.values()],
        f'F1 Score (Test) Clase {i}': [f1[i] for f1 in f1_scores.values()]
    })
    metricas_clase.set_index('Modelos', inplace=True)
    metricas_clases.append(metricas_clase)

def get_metricas_detalladas_por_clase(clase):
    if clase < 0 or clase >= classes:
        raise ValueError("Número de clase inválido.")
    return metricas_clases[clase]

In [ ]:
metricas_generales

,Mejores Parámetros,Mejor Score (Train),Accuracy (Test),Sobreajuste
Modelos,,,,
Regresión Logística,"{'C': 10, 'penalty': 'l2'}",0.678257,0.679580,0.010006
KNN,{'n_neighbors': 53},0.664130,0.650615,0.034780
Árbol de Decisión,{'max_depth': 9},0.688033,0.686097,0.096923
Bosques Aleatorios,"{'max_depth': 97, 'n_estimators': 200}",0.742354,0.737509,0.262491
Bagging,"{'max_samples': 900, 'n_estimators': 100}",0.728696,0.735337,0.089900
Pasting,"{'max_samples': 700, 'n_estimators': 100}",0.726213,0.728458,0.083431


In [ ]:
clase = 0
metricas_clase_especifica = get_metricas_detalladas_por_clase(clase)
metricas_clase_especifica

,Precision (Test) Clase 0,Recall (Test) Clase 0,F1 Score (Test) Clase 0
Modelos,,,
Regresión Logística,0.697917,0.718017,0.707824
KNN,0.663164,0.718687,0.689810
Árbol de Decisión,0.704308,0.722706,0.713388
Bosques Aleatorios,0.741509,0.789685,0.764839
Bagging,0.733170,0.802411,0.766230
Pasting,0.726940,0.797053,0.760383


In [ ]:
clase = 1
metricas_clase_especifica = get_metricas_detalladas_por_clase(clase)
metricas_clase_especifica

,Precision (Test) Clase 1,Recall (Test) Clase 1,F1 Score (Test) Clase 1
Modelos,,,
Regresión Logística,0.656607,0.634358,0.645291
KNN,0.632867,0.570528,0.600083
Árbol de Decisión,0.663415,0.643026,0.653061
Bosques Aleatorios,0.732082,0.676123,0.702991
Bagging,0.738475,0.656422,0.695035
Pasting,0.730667,0.647754,0.686717


## 6.1.2. Con las características más relacionadas

### A. Validación cruzada

#### Mejor modelo de Regresión Logística

In [ ]:
param_grid_reg_clf_filtro_bi = [{'penalty': ['l1', 'l2'],
                           'C': [0.01, 0.1, 1.0, 10, 100]}]

In [ ]:
grid_reg_clf_filtro_bi = LogisticRegression(solver='liblinear')

grid_search_reg_clf_filtro_bi = GridSearchCV(grid_reg_clf_filtro_bi, param_grid_reg_clf_filtro_bi, cv=5, scoring='accuracy')

grid_search_reg_clf_filtro_bi.fit(X3_filtro_train, y3_filtro_train)

GridSearchCV(cv=5, estimator=LogisticRegression(solver='liblinear'),
             param_grid=[{'C': [0.01, 0.1, 1.0, 10, 100],
                          'penalty': ['l1', 'l2']}],
             scoring='accuracy')

#### Mejor modelo para KNN

In [ ]:
X3_filtro_train.shape

(6443, 83)

In [ ]:
param_grid_knn_clf_filtro_bi = [{'n_neighbors': list(range(2, 83))}]

In [ ]:
grid_knn_clf_filtro_bi = KNeighborsClassifier()

grid_search_knn_clf_filtro_bi = GridSearchCV(grid_knn_clf_filtro_bi, param_grid_knn_clf_filtro_bi, cv=5, scoring='accuracy')

grid_search_knn_clf_filtro_bi.fit(X3_filtro_train, y3_filtro_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15, 16, 17, 18, 19, 20, 21,
                                          22, 23, 24, 25, 26, 27, 28, 29, 30,
                                          31, ...]}],
             scoring='accuracy')

#### Mejor modelo para Árbol de Decisión

In [ ]:
param_grid_tree_clf_filtro_bi = [{'max_depth': list(range(1, 83))}]

In [ ]:
grid_tree_clf_filtro_bi = DecisionTreeClassifier(random_state=42)

grid_search_tree_clf_filtro_bi = GridSearchCV(grid_tree_clf_filtro_bi, param_grid_tree_clf_filtro_bi, cv=5, scoring='accuracy')

grid_search_tree_clf_filtro_bi.fit(X3_filtro_train, y3_filtro_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid=[{'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                        13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                        23, 24, 25, 26, 27, 28, 29, 30, ...]}],
             scoring='accuracy')

#### Mejor modelo para Bosques Aleatorios

In [ ]:
param_grid_rndf_clf_filtro_bi = [{'n_estimators': list(range(100, 301, 100))},
                       {'n_estimators': list(range(100, 301, 100)), 'max_depth': list(range(1, 83))}]

In [ ]:
grid_rndf_clf_filtro_bi = RandomForestClassifier()

grid_search_rndf_clf_filtro_bi = GridSearchCV(grid_rndf_clf_filtro_bi, param_grid_rndf_clf_filtro_bi, cv=5, scoring='accuracy')

grid_search_rndf_clf_filtro_bi.fit(X3_filtro_train, y3_filtro_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid=[{'n_estimators': [100, 200, 300]},
                         {'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                        13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                        23, 24, 25, 26, 27, 28, 29, 30, ...],
                          'n_estimators': [100, 200, 300]}],
             scoring='accuracy')

#### Mejor modelo para Aprendizaje Ensamblado

Método de conjunto Bagging:

In [ ]:
param_grid_bag_clf_filtro_bi = [{'n_estimators': list(range(50, 101, 50)),
                      'max_samples': list(range(500, 901, 200))}]

In [ ]:
grid_bag_clf_filtro_bi = BaggingClassifier(DecisionTreeClassifier(), bootstrap=True, random_state=42)

grid_search_bag_clf_filtro_bi = GridSearchCV(grid_bag_clf_filtro_bi, param_grid_bag_clf_filtro_bi, cv=5, scoring='accuracy')

grid_search_bag_clf_filtro_bi.fit(X3_filtro_train, y3_filtro_train)

GridSearchCV(cv=5,
             estimator=BaggingClassifier(estimator=DecisionTreeClassifier(),
                                         random_state=42),
             param_grid=[{'max_samples': [500, 700, 900],
                          'n_estimators': [50, 100]}],
             scoring='accuracy')

Método de conjunto Pasting:

In [ ]:
param_grid_pas_clf_filtro_bi = [{'n_estimators': list(range(50, 101, 50)),
                       'max_samples': list(range(500, 901, 200))}]

In [ ]:
grid_pas_clf_filtro_bi = BaggingClassifier(DecisionTreeClassifier(), bootstrap=False, random_state=42)

grid_search_pas_clf_filtro_bi = GridSearchCV(grid_pas_clf_filtro_bi, param_grid_pas_clf_filtro_bi, cv=5, scoring='accuracy')

grid_search_pas_clf_filtro_bi.fit(X3_filtro_train, y3_filtro_train)

GridSearchCV(cv=5,
             estimator=BaggingClassifier(bootstrap=False,
                                         estimator=DecisionTreeClassifier(),
                                         random_state=42),
             param_grid=[{'max_samples': [500, 700, 900],
                          'n_estimators': [50, 100]}],
             scoring='accuracy')

### B. Mejores hiperparámetros, puntajes y estimadores

In [ ]:
best_params_reg_clf_filtro_bi = grid_search_reg_clf_filtro_bi.best_params_
best_params_knn_clf_filtro_bi = grid_search_knn_clf_filtro_bi.best_params_
best_params_tree_clf_filtro_bi = grid_search_tree_clf_filtro_bi.best_params_
best_params_rndf_clf_filtro_bi = grid_search_rndf_clf_filtro_bi.best_params_
best_params_bag_clf_filtro_bi = grid_search_bag_clf_filtro_bi.best_params_
best_params_pas_clf_filtro_bi = grid_search_pas_clf_filtro_bi.best_params_

In [ ]:
best_score_reg_clf_filtro_bi = grid_search_reg_clf_filtro_bi.best_score_
best_score_knn_clf_filtro_bi = grid_search_knn_clf_filtro_bi.best_score_
best_score_tree_clf_filtro_bi = grid_search_tree_clf_filtro_bi.best_score_
best_score_rndf_clf_filtro_bi = grid_search_rndf_clf_filtro_bi.best_score_
best_score_bag_clf_filtro_bi = grid_search_bag_clf_filtro_bi.best_score_
best_score_pas_clf_filtro_bi = grid_search_pas_clf_filtro_bi.best_score_

In [ ]:
best_estimator_reg_clf_filtro_bi = grid_search_reg_clf_filtro_bi.best_estimator_
best_estimator_knn_clf_filtro_bi = grid_search_knn_clf_filtro_bi.best_estimator_
best_estimator_tree_clf_filtro_bi = grid_search_tree_clf_filtro_bi.best_estimator_
best_estimator_rndf_clf_filtro_bi = grid_search_rndf_clf_filtro_bi.best_estimator_
best_estimator_bag_clf_filtro_bi = grid_search_bag_clf_filtro_bi.best_estimator_
best_estimator_pas_clf_filtro_bi = grid_search_pas_clf_filtro_bi.best_estimator_

### C. Evaluación de los modelos

In [ ]:
best_params_filtro = {
    'Regresión Logística': best_params_reg_clf_filtro_bi,
    'KNN': best_params_knn_clf_filtro_bi,
    'Árbol de Decisión': best_params_tree_clf_filtro_bi,
    'Bosques Aleatorios': best_params_rndf_clf_filtro_bi,
    'Bagging': best_params_bag_clf_filtro_bi,
    'Pasting': best_params_pas_clf_filtro_bi
}

best_scores_filtro = {
    'Regresión Logística': best_score_reg_clf_filtro_bi,
    'KNN': best_score_knn_clf_filtro_bi,
    'Árbol de Decisión': best_score_tree_clf_filtro_bi,
    'Bosques Aleatorios': best_score_rndf_clf_filtro_bi,
    'Bagging': best_score_bag_clf_filtro_bi,
    'Pasting': best_score_pas_clf_filtro_bi
}

best_estimators_filtro = {
    'Regresión Logística': best_estimator_reg_clf_filtro_bi,
    'KNN': best_estimator_knn_clf_filtro_bi,
    'Árbol de Decisión': best_estimator_tree_clf_filtro_bi,
    'Bosques Aleatorios': best_estimator_rndf_clf_filtro_bi,
    'Bagging': best_estimator_bag_clf_filtro_bi,
    'Pasting': best_estimator_pas_clf_filtro_bi
}

y_pred_filtro = {
    'Regresión Logística': best_estimator_reg_clf_filtro_bi.predict(X3_filtro_test),
    'KNN': best_estimator_knn_clf_filtro_bi.predict(X3_filtro_test),
    'Árbol de Decisión': best_estimator_tree_clf_filtro_bi.predict(X3_filtro_test),
    'Bosques Aleatorios': best_estimator_rndf_clf_filtro_bi.predict(X3_filtro_test),
    'Bagging': best_estimator_bag_clf_filtro_bi.predict(X3_filtro_test),
    'Pasting': best_estimator_pas_clf_filtro_bi.predict(X3_filtro_test)
}

precision_scores_filtro = {}
recall_scores_filtro = {}
f1_scores_filtro = {}

for model, y_pred_test in y_pred_filtro.items():
    precision, recall, f1, _ = precision_recall_fscore_support(y3_filtro_test, y_pred_test, average=None, zero_division=1)
    precision_scores_filtro[model] = precision
    recall_scores_filtro[model] = recall
    f1_scores_filtro[model] = f1

sobreajuste_filtro = {}
for model, estimator in best_estimators_filtro.items():
    score_train = estimator.score(X3_filtro_train, y3_filtro_train)
    score_test = estimator.score(X3_filtro_test, y3_filtro_test)
    sobreajuste_filtro[model] = abs(score_train - score_test)

metricas_generales_filtro = pd.DataFrame({
    'Modelos': list(best_params_filtro.keys()),
    'Mejores Parámetros': list(best_params_filtro.values()),
    'Mejor Score (Train)': list(best_scores_filtro.values()),
    'Accuracy (Test)': [accuracy_score(y3_filtro_test, y_pred_filtro[model]) for model in best_params_filtro.keys()],
    'Sobreajuste': list(sobreajuste_filtro.values())
})

metricas_generales_filtro.set_index('Modelos', inplace=True)

metricas_clases_filtro = []
classes = len(set(y3_test))

for i in range(classes):
    metricas_clase_filtro = pd.DataFrame({
        'Modelos': list(best_params_filtro.keys()),
        f'Precision (Test) Clase {i}': [precision[i] for precision in precision_scores_filtro.values()],
        f'Recall (Test) Clase {i}': [recall[i] for recall in recall_scores_filtro.values()],
        f'F1 Score (Test) Clase {i}': [f1[i] for f1 in f1_scores_filtro.values()]
    })
    metricas_clase_filtro.set_index('Modelos', inplace=True)
    metricas_clases_filtro.append(metricas_clase_filtro)

def get_metricas_detalladas_por_clase_filtro(clase):
    if clase < 0 or clase >= classes:
        raise ValueError("Número de clase inválido.")
    return metricas_clases_filtro[clase]

In [ ]:
metricas_generales_filtro

,Mejores Parámetros,Mejor Score (Train),Accuracy (Test),Sobreajuste
Modelos,,,,
Regresión Logística,"{'C': 100, 'penalty': 'l1'}",0.674999,0.673063,0.012487
KNN,{'n_neighbors': 35},0.664596,0.642650,0.048332
Árbol de Decisión,{'max_depth': 9},0.692068,0.694062,0.082905
Bosques Aleatorios,"{'max_depth': 51, 'n_estimators': 300}",0.741269,0.736061,0.263939
Bagging,"{'max_samples': 900, 'n_estimators': 100}",0.726989,0.727734,0.097814
Pasting,"{'max_samples': 900, 'n_estimators': 50}",0.729938,0.730992,0.095021


In [ ]:
clase = 0
metricas_clase_especifica = get_metricas_detalladas_por_clase_filtro(clase)
metricas_clase_especifica

,Precision (Test) Clase 0,Recall (Test) Clase 0,F1 Score (Test) Clase 0
Modelos,,,
Regresión Logística,0.688860,0.720697,0.704419
KNN,0.660533,0.697254,0.678397
Árbol de Decisión,0.708494,0.737441,0.722678
Bosques Aleatorios,0.740252,0.788346,0.763542
Bagging,0.731418,0.784327,0.756949
Pasting,0.724282,0.811119,0.765245


In [ ]:
clase = 1
metricas_clase_especifica = get_metricas_detalladas_por_clase_filtro(clase)
metricas_clase_especifica

,Precision (Test) Clase 1,Recall (Test) Clase 1,F1 Score (Test) Clase 1
Modelos,,,
Regresión Logística,0.652500,0.617021,0.634265
KNN,0.618887,0.578408,0.597963
Árbol de Decisión,0.675497,0.643026,0.658862
Bosques Aleatorios,0.730375,0.674547,0.701352
Bagging,0.722653,0.661151,0.690535
Pasting,0.741284,0.636722,0.685036
